<a href="https://colab.research.google.com/github/Kannapat-Tich/DataScience-Learning/blob/main/02-CreditScore_Customers/01-DataPreparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
# !pip install openpyxl

# My old dataset

## customer_master

### 1. Data

In [ ]:
import pandas as pd

customer = pd.read_csv('/content/drive/MyDrive/PEA-intern/CreditScore/data/1.customer_master_b.csv')
customer = pd.DataFrame(customer)
# Print
customer

In [ ]:
customer.info()

In [ ]:
customer.rename(columns={"customer_name":"ca_name"},inplace=True)

In [ ]:
customer.columns

#### 1.1 Columns

##### bp_type
ประเภทคู่ค้าทางธุรกิจ

In [ ]:
customer['bp_type'].isnull().sum()

In [ ]:
customer['bp_type'].value_counts()

#### act_class
คลาสบัญชี

In [ ]:
customer['act_class'].value_counts()

In [ ]:
customer['act_class'].isnull().sum()

#### flg_bs
ประเภทผู้ใช้ไฟ

In [ ]:
customer['flg_bs'].value_counts()

#### mi_date

In [ ]:
customer['mi_date'].isnull().sum()

#### mo_date

In [ ]:
customer['mo_date'].isnull().sum()

#### kva
ขนาดหม้อแปลง

In [ ]:
customer['kva'].isnull().sum()

#### meter_type

In [ ]:
customer['meter_type'].value_counts()

In [ ]:
customer['meter_type'].isnull().sum()

#### meter_size

In [ ]:
customer['meter_size'].value_counts()

In [ ]:
customer['meter_size'].isnull().sum()

#### installation_type

In [ ]:
customer['installation_type'].value_counts()

In [ ]:
customer['installation_type'].isnull().sum()

#### portion
สัดส่วน

In [ ]:
customer['portion'].value_counts()

In [ ]:
customer['portion'].isnull().sum()

#### volt_buy

In [ ]:
customer['volt_buy'].value_counts()

In [ ]:
customer['volt_buy'].isnull().sum()

#### ratecat
อัตราคิดเงินค่าไฟ

In [ ]:
customer['ratecat'].value_counts()

In [ ]:
customer['ratecat'].isnull().sum()

#### sec_dep_cash
เงินประกัน เงินสด

In [ ]:
customer['sec_dep_cash'].isnull().sum()

#### sec_dep_noncash
เงินประกัน ที่ไม่ใช่เงินสด

In [ ]:
customer['sec_dep_noncash'].isnull().sum()

#### 1.2 Features
คอลัมน์ที่มองว่าสำคัญต่อการนำมาคำนวณการให้เครดิตลูกค้า

* bp_type
    * ดูพฤติกรรมของลูกค้าทั้งสองทั้งกลุ่มว่ามีพฤติกรรมเหมือนหรือแตกต่างกันอย่างไร
* act_class
    * ดูพฤติกรรมด้านการชำระเงินของแต่ละคลาสบัญชี
* mi_date & mo_date
    * นำมาใช้ในการคำนวณอายุการเป็นลูกค้า
* kva, volt_buy & portion
    * ปริมาณการใช้ไฟฟ้า อาจมีผลต่อความเป็นลูกค้า
* meter_type & meter_size
    * ข้อมูลเกี่ยวกับการติดตั้งมิเตอร์
* ratecat
    * อัตราคิดเงินค่าไฟฟ้าที่ลูกค้าใช้
* region
    * ดูพฤติกรรมของลูกค้าตามภูมิภาค
* sec_dep
    * ข้อมูลเกี่ยวกับเงินประกันที่ลูกค้าจ่าย

### 2. Data Cleansing

In [ ]:
customer_clean = customer

In [ ]:
customer_clean.info()

##### 2.1 Cleansing mi_date / mo_date by CA
* Drop Duplicated data

In [ ]:
customer_clean.ca.loc[customer_clean.ca.duplicated()]

In [ ]:
customer_clean.ca.value_counts()

In [ ]:
test = customer_clean.iloc[1:3].reset_index(drop=True)
# Print
test

In [ ]:
# ดรอปแถวที่ไม่ใช่แถวสุดท้าย
last = test.iloc[[-1]]
# แทนที่ค่า mi_date ในแถวสุดท้ายด้วยค่า mi_date จากแถวแรก
last.loc[last.index, 'mi_date'] = test.iloc[0]['mi_date']
# Print
last

In [ ]:
test2 = customer_clean.loc[customer_clean['ca'] == '020003654896']
# Print
test2

In [ ]:
test3 = customer_clean.loc[customer_clean['ca'] == '020003658628']
# Print
test3

In [ ]:
test3.head(15)

### Summary : Why CA have duplicated data ?
CA ซ้ำมีสองกรณี ดังนี้
1. สองแถว
    * วันที่ move in / move out มีความต่อเนื่องกัน
2. มากกว่า 2 แถว
    * มีการเปลี่ยนแปลง meter size
        * ทำให้วันที่ติดตั้งมิเตอร์เปลี่ยนแปลง
        * ทำให้ portion เปลี่ยนแปลง

#### 2.2 Drop Duplicated

###### Spliting DataFrame

In [ ]:
# Split data
    # Values_count
dup_counts = customer_clean['ca'].value_counts()[customer_clean['ca'].value_counts() > 1]
    # DataFrame
dup = customer_clean.loc[customer_clean['ca'].isin(dup_counts.index)].reset_index(drop=True)
notdup = customer_clean.loc[~customer_clean['ca'].isin(dup_counts.index)].reset_index(drop=True)

In [ ]:
dup_counts

In [ ]:
notdup

In [ ]:
dup

In [ ]:
dup = dup.sort_values(by='ca').reset_index(drop=True)
# Print
dup

In [ ]:
dup = dup.sort_values(by=['ca','mi_date','mo_date']).reset_index(drop=True)
# Print
dup

##### mi_date

In [ ]:
dup_2024 = dup[dup['mi_date'].str.startswith('2024')]
dup_2024

In [ ]:
notdup_24 = notdup[notdup.mi_date.str.startswith('2024')]
notdup_24

##### Solution 1

##### Fuction

In [ ]:
dup['index'] = dup.groupby('ca').cumcount()
# Print
dup

In [ ]:
max_index_rows = dup.groupby('ca')['index'].idxmax()
max_index_rows

In [ ]:
dup[dup.ca == '020000054768']

In [ ]:
max_index_values = dup.loc[max_index_rows, 'mi_date'].values
max_index_values

In [ ]:
def replace(df,col):

    max_index_rows = df.groupby('ca')['index'].idxmax()

    # ดึงค่าจากแถวที่มีค่า index มากที่สุดในแต่ละกลุ่ม
    max_index_values = df.loc[max_index_rows, col].values

    # หา index ของแถวที่มีค่า index น้อยที่สุดในแต่ละกลุ่ม ca
    min_index_rows = df.groupby('ca')['index'].idxmin()

    # แทนที่ค่าใน col ของแถวที่มี index มากที่สุดด้วยค่าจากแถวที่มี index น้อยที่สุด
    df.loc[max_index_rows, col] = df.loc[min_index_rows, col].values

    # ลบแถวที่ซ้ำกันที่คอลัมน์ ca และเก็บแถวสุดท้าย
    df = df.drop_duplicates(subset='ca', keep='last')

    # รีเซ็ต index
    df.reset_index(drop=True, inplace=True)

    return df

In [ ]:
dup_test = replace(dup,'mi_date')

In [ ]:
dup_test

In [ ]:
dup_test.ca.duplicated().sum()

In [ ]:
customer_df = pd.concat([dup_test, notdup]).reset_index(drop=True)
# Print
customer_df

In [ ]:
customer_df.ca.duplicated().sum()

In [ ]:
customer_df.drop(columns=['index'],inplace=True)
customer_df

##### Datetime

In [ ]:
customer_df.mi_date = pd.to_datetime(customer_df.mi_date,errors='coerce')
customer_df.info()

In [ ]:
customer_df.dropna(subset=['mi_date'],inplace=True)
customer_df.info()

In [ ]:
customer_df.mo_date = pd.to_datetime(customer_df.mo_date,errors='coerce')
customer_df.info()

In [ ]:
customer_df.mo_date.fillna('2023-12-31',inplace=True)
customer_df

##### Meter_age

In [ ]:
mo = customer_df.mo_date[:1]
mi = customer_df.mi_date[:1]
print(mo,'\n', mi)
test = (mo - mi).dt.days
print(test)

In [ ]:
def meter_age(df):
    mo = df.mo_date
    mi = df.mi_date
    df['meter_age'] = (mo - mi).dt.days
    return df

In [ ]:
customer_df = meter_age(customer_df)
# Print
customer_df

In [ ]:
min(customer_df.meter_age)

In [ ]:
customer_df.loc[customer_df.meter_age < 0]

In [ ]:
customer_df.loc[customer_df.meter_age == 0]

In [ ]:
notdup[notdup.ca=='020017292631']

In [ ]:
notdup[notdup.ca=='020017769897']

##### Solution 2

In [ ]:
dup.head(15)

In [ ]:
dup.ca.value_counts()

In [ ]:
dup[dup.ca =='020003654896']

##### Function

In [ ]:
def call_ca(ca):
    return dup[dup.ca == ca]

In [ ]:
call_ca('020000054768')

In [ ]:
max_index = dup.groupby('ca')['index'].idxmax()
max_index

In [ ]:
def call_ca(ca):
    return dup[dup.ca == ca]

def meter_age(ca):
    # Call max & min from each of ca groups
    for i in call_ca(ca):
        max_index = i.loc[i['index'].idxmax()]
        max_values = i.loc[max_index, 'mi_date'].values
        min_index = i.loc[i['index'].idxmin()]
        min_date = i.loc[min_index, 'mi_date']
    # Replace the 'mi_date' in the original DataFrame for the row with the maximum index
    ca_input.loc[df['index'] == max_idx, 'mi_date'] = min_date

    # Drop Duplicate
    df = ca_input.drop_duplicates(subset='ca', keep='last')
    df.reset_index(drop=True, inplace=True)
    return df

In [ ]:
ca = '020000054768'

##### Solution 3

###### Test

In [ ]:
dup = dup.sort_values(by=['ca','mi_date']).reset_index(drop=True)
dup

In [ ]:
notdup

In [ ]:
dup.ca.value_counts()

In [ ]:
mask = dup[dup.ca == '020015588070']
mask

In [ ]:
first = mask.head(1)
first

In [ ]:
first = mask.iloc[0]
last = mask.iloc[-1]

In [ ]:
first

In [ ]:
last

In [ ]:
for i in mask:
    # Call Rows
    first = mask.iloc[0]
    last = mask.iloc[-1]
    # Call Values
    min_date = first.mi_date
    # Replace max_date with min_date
    mask.at[mask.index[-1], 'mi_date'] = min_date
    # Drop others
    mask = mask.drop_duplicates(subset='ca',keep='last')

In [ ]:
mask

##### Function

In [ ]:
def new_dup_df(dup,ca_input):

    mask_ca = dup[dup['ca'] == ca_input]

    # Call Rows
    first = mask_ca.iloc[0]
    # Call Values
    min_date = first['mi_date']
    # Replace max_date with min_date
    mask_ca.at[mask_ca.index[-1], 'mi_date'] = min_date
    # Drop others
    mask_ca = mask_ca.drop_duplicates(subset='ca', keep='last')
    # Reset Index
    mask_ca = mask_ca.reset_index(drop=True)

    # Update dup DF
    dup[dup.ca == ca_input] = dup[dup['ca'] == ca_input].drop_duplicates(subset=['ca'],keep='last')
    dup = dup.dropna(how='all')
    dup = dup.reset_index(drop=True)
    # Setting index
    mask_ca.set_index('ca', inplace=True)
    dup.set_index('ca', inplace=True)

    # Update `dup` DataFrame with values from `update`
    dup.loc[mask_ca.index] = mask_ca

    # Reset index
    dup.reset_index(inplace=True)
    mask_ca.reset_index(inplace=True)

    return dup

##### Cal by Function

In [ ]:
# # :10,000

# ca = dup['ca']

# while True:
#     counts = ca[:10001].value_counts()
#     has_duplicates = False

#     for value, count in counts.items():
#         if count > 1:
#             dup = new_dup_df(dup, value)
#             ca = dup['ca']
#             has_duplicates = True
#             break

#     if not has_duplicates:
#         break

# counts

In [ ]:
# # :20,000

# ca = dup['ca']

# while True:
#     counts = ca[10000:20001].value_counts()
#     has_duplicates = False

#     for value, count in counts.items():
#         if count > 1:
#             dup = new_dup_df(dup, value)
#             ca = dup['ca']
#             has_duplicates = True
#             break

#     if not has_duplicates:
#         break

# counts

In [ ]:
# # :30,000

# ca = dup['ca']

# while True:
#     counts = ca[20000:30001].value_counts()
#     has_duplicates = False

#     for value, count in counts.items():
#         if count > 1:
#             dup = new_dup_df(dup, value)
#             ca = dup['ca']
#             has_duplicates = True
#             break

#     if not has_duplicates:
#         break

# counts

In [ ]:
# # :40,000

# ca = dup['ca']

# while True:
#     counts = ca[30000:40001].value_counts()
#     has_duplicates = False

#     for value, count in counts.items():
#         if count > 1:
#             dup = new_dup_df(dup, value)
#             ca = dup['ca']
#             has_duplicates = True
#             break

#     if not has_duplicates:
#         break

# counts

In [ ]:
# # :50,000

# ca = dup['ca']

# while True:
#     counts = ca[40000:50001].value_counts()
#     has_duplicates = False

#     for value, count in counts.items():
#         if count > 1:
#             dup = new_dup_df(dup, value)
#             ca = dup['ca']
#             has_duplicates = True
#             break

#     if not has_duplicates:
#         break

# counts

#### 2.3 Merge

###### Recheck

In [ ]:
# save
dup.to_csv('dup.csv')

In [ ]:
import pandas as pd
dup = pd.read_csv('/content/dup.csv')
dup = pd.DataFrame(dup)
dup

In [ ]:
dup.info()

In [ ]:
dup.columns

In [ ]:
dup.drop(columns=['Unnamed: 0','index'],axis=1,inplace=True)
dup

In [ ]:
dup.info()

In [ ]:
notdup

In [ ]:
notdup.ca.duplicated().sum()

In [ ]:
# notdup.set_index('ca',inplace=True)
notdup.reset_index(inplace=True)
notdup

In [ ]:
notdup.columns

In [ ]:
notdup.drop(columns='index',inplace=True)
notdup

In [ ]:
notdup.info()

In [ ]:
dup.info()

##### concat

In [ ]:
customer = pd.concat([dup, notdup]).reset_index(drop=True)
# Print
customer

In [ ]:
# Save
customer.to_csv('customer.csv')

#### 2.4 Datetime

In [ ]:
customer

#### mi_date

In [ ]:
customer.mi_date = pd.to_datetime(customer.mi_date,errors='coerce')
customer.info()

In [ ]:
customer.mi_date.isnull().sum()

In [ ]:
customer.dropna(subset=['mi_date'],inplace=True)
customer.mi_date.isnull().sum()

#### mo_date

In [ ]:
customer.mo_date = pd.to_datetime(customer.mo_date,errors='coerce')
customer.info()

In [ ]:
customer

In [ ]:
customer.mo_date.fillna('2024-12-31',inplace=True)
customer

#### meter_age

In [ ]:
mi = customer.mi_date
mo = customer.mo_date
customer['meter_age'] = (mo-mi).dt.days

customer

In [ ]:
customer[customer.meter_age < 0]

In [ ]:
customer.info()

In [ ]:
# Save
customer.to_csv(
    '/content/drive/MyDrive/PEA-intern/CreditScore/df/customer.csv',
    index=False
    )

## customer_payment_behavior

## 1. Data

In [ ]:
import pandas as pd
payment = pd.read_csv('/content/drive/MyDrive/PEA-intern/CreditScore/data/2.customer_payment_behavior2023.csv')
payment = pd.DataFrame(payment)

payment.sort_values(by=['CA','BILL_PERIOD','BILL_DUEDATE'],inplace=True)
payment=payment.reset_index(drop=True)
payment

In [ ]:
payment.info()

In [ ]:
payment.CA.value_counts()

In [ ]:
payment[payment.CA == 20015991658]

### 1.1 Columns

In [ ]:
payment.columns

#### KTOKL / act_class

In [ ]:
payment.KTOKL.value_counts()

#### TARIFTYP / ratecat

In [ ]:
payment.TARIFTYP.value_counts()

#### BILL_PERIOD
บิลเดือน

In [ ]:
payment.BILL_PERIOD.value_counts()

#### BILL_DUEDATE
วันที่ครบกำหนดชำระ

In [ ]:
payment.BILL_DUEDATE.value_counts()

#### BILL_PAYDATE
วันที่รับชำระ

In [ ]:
payment.BILL_PAYDATE.value_counts()

#### AMT_DEBT
จำนวนเงินยอดตั้งหนี้

In [ ]:
payment.AMT_DEBT.value_counts()

#### INSTALLMENT
ผ่อนชำระ mark "X" เมื่อมีแผนผ่อนชำระ

In [ ]:
payment.INSTALLMENT.value_counts()

#### INST_PERIOD
จำนวนงวดผ่อนชำระ

In [ ]:
payment.INST_PERIOD.value_counts()

#### PAYMENT_CH
ช่องทางชำระเงิน

In [ ]:
payment.PAYMENT_CH.value_counts()

#### TSIC_ID

In [ ]:
payment.TSIC_ID.value_counts()

### 1.2 Features

* KTOKL / act_class
    * ดูพฤติกรรมด้านการชำระเงินของแต่ละคลาสบัญชี

* TARIFTYP / ratecat
    * อัตราคิดเงินค่าไฟของลูกค้า

* BILL_PERIOD ,BILL_DUEDATE ,BILL_PAYDATE
    * ดูพฤติกรรมการชำระเงินของลูกค้า

* AMT_DEBT ,INSTALLMENT ,INST_PERIOD
    * พฤติกรรมหรือความสามารถการชำระหนี้ของลูกค้าก
* PAYMENT_CH
    * การเข้าถึงบริการการชำระเงินของลูกค้า
* TSIC_ID
    * ศึกษาพฤติกรรมของลูกค้าตามแต่ละกลุ่มอุตสหากรรม

## 2. Data Cleansing

In [ ]:
payment.columns = payment.columns.str.lower()
payment.info()

### change Column Name

In [ ]:
payment.rename(columns={"ktokl":"act_class","tariftyp":"ratecat",
                        "tsic_id":"tsic"},inplace=True)
payment.info()

### Convert type

In [ ]:
payment.ca = payment.ca.astype(str)
payment.bp = payment.bp.astype(str)
payment.act_class = payment.act_class.astype("category")
payment.ratecat = payment.ratecat.astype("category")
payment.bill_duedate = payment.bill_duedate.astype(str)
payment.bill_paydate = payment.bill_paydate.astype(str)
payment.tsic = payment.tsic.astype("category")
payment.info()

In [ ]:
payment

In [ ]:
payment.bill_duedate = pd.to_datetime(payment.bill_duedate)
payment.info()

In [ ]:
payment.bill_paydate = pd.to_datetime(payment.bill_paydate)
payment.info()

In [ ]:
payment

### Missing Data

In [ ]:
payment.installment.fillna('O',inplace=True)
payment

In [ ]:
payment.installment = payment.installment.astype("category")
payment.info()

### Duplicates Checking

In [ ]:
payment.ca.value_counts()

In [ ]:
payment.columns

In [ ]:
# Drop all
col = ['trsg', 'ca', 'opbel', 'xblnr', 'ca_name', 'act_class', 'ratecat',
       'tsic', 'bill_period', 'bill_duedate', 'bill_paydate', 'amt_debt',
       'installment', 'inst_period', 'bp', 'tax_id', 'branch', 'dept_doc_type',
       'pay_doc_type', 'payment_ch', 'etl_datetime']
payment.drop_duplicates(subset=col,inplace=True)
payment.reset_index(drop=True,inplace=True)
payment.ca.value_counts()

In [ ]:
payment[(payment.ca == '20017893613')]

In [ ]:
payment.iloc[1127337]

In [ ]:
payment.iloc[1127338]

In [ ]:
payment.iloc[1127339]

In [ ]:
payment.iloc[1127340]

In [ ]:
payment[payment['ca'] == '20017893613']

In [ ]:
payment[payment.ca == '20008920531']

In [ ]:
payment[(payment.ca =='20005832310')]

In [ ]:
payment.iloc[529121]

In [ ]:
payment.iloc[529122]

#### Summary Duplicate Data
มีข้อมูลซ้ำซ้อนสองกรณี
* แบบที่ซ้ำซ้อนทุกคอลัมน์ สามารถลบเหลือแถวเดียวได้
* แบบซ้ำซ้อนเพราะมีการจ่ายที่ไม่เหมือนกัน
    * บาง bill_period มีการจ่ายมากกว่า 1 ครั้ง เพราะขึ้นอยู่กับรูปแบบการชำระเงิน (payment_ch)

In [ ]:
payment.dept_doc_type.value_counts()

In [ ]:
payment.pay_doc_type.value_counts()

In [ ]:
payment.payment_ch.value_counts()

In [ ]:
payment.branch.value_counts()

#### payment type

In [ ]:
payment.pay_doc_type.unique()

In [ ]:
payment.payment_ch.unique()

In [ ]:
col = ['pay_doc_type','payment_ch']
pay_type = pd.DataFrame(payment[col])
pay_type.drop_duplicates(inplace=True)
pay_type.reset_index(drop=True,inplace=True)

pay_type

In [ ]:
pay_type.sort_values(by=['payment_ch'],inplace=True)
pay_type.reset_index(drop=True,inplace=True)

pay_type

In [ ]:
pay_type.head(30)

In [ ]:
pay_type.tail(35)

In [ ]:
ca_counts = pd.DataFrame(payment.ca.value_counts())
ca_counts.reset_index(inplace=True)
ca_counts

In [ ]:
ca_counts[(ca_counts['count'] < 12)]

#### Checking doc type

#####  สลับกันในเดือนเดียว

PE / MG

In [ ]:
payment[(payment.ca == '20017893613')]

PH / MP

In [ ]:
payment[(payment.ca=='20001831889')]

##### มีประเภทอื่นแทรกมาแบบสุ่ม

In [ ]:
payment[(payment.ca=='20008920531')]

##### มีประเภทเดียว และมีซ้ำกันมากกว่า 1 เดือน (มากกว่า 13 แถว)

In [ ]:
def doc(row_input):
    row_input = ca_counts.iloc[row_input]
    ca = row_input['ca']
    return payment[(payment.ca==ca)]

###### PA

In [ ]:
payment[(payment.ca=='20005832310')]

In [ ]:
payment.iloc[529131]

In [ ]:
payment.iloc[529132]

In [ ]:
payment[(payment.ca=='20005832505')]

In [ ]:
payment[(payment.ca=='20005661993')]

###### PC

In [ ]:
payment[(payment.ca=='20006042604')]

In [ ]:
payment[(payment.ca=='20006526570')]

###### PB

In [ ]:
payment[(payment.ca=='20010631872')]

In [ ]:
payment[(payment.ca=='20010632158')]

In [ ]:
payment[(payment.ca=='20024676141')]


###### PE

In [ ]:
payment[(payment.ca=='20012307126')]

###### MY

In [ ]:
payment[(payment.ca=='20023088514')]

###### OZ

In [ ]:
doc(70)

###### MT

In [ ]:
doc(75)

##### มีประเภทเดียว และจ่ายสองครั้งแค่เดือนเดียว (13 แถว)

###### PH

In [ ]:
doc(300)

###### PI

In [ ]:
doc(400)

###### MD

In [ ]:
doc(200)

###### PG

In [ ]:
doc(720)

###### PA

In [ ]:
doc(730)

##### จ่ายตามรอบเดือน (12 แถว)

###### PI

In [ ]:
doc(2100)

###### PA

In [ ]:
doc(1800)

###### MG

In [ ]:
doc(176963)

###### PK

In [ ]:
doc(732)

###### MD

In [ ]:
doc(740)

###### PH

In [ ]:
doc(750)

###### OZ

In [ ]:
doc(850)

###### MY

In [ ]:
doc(900)

###### PB

In [ ]:
doc(1000)

###### PF

In [ ]:
doc(1003)

###### PL

In [ ]:
doc(1300)

###### PG

In [ ]:
doc(1400)

##### จ่ายตามรอบเดือน แต่เปลี่ยนเป็นประเภทอื่น

###### PG / MY

In [ ]:
doc(1600)

###### PA / MY

In [ ]:
doc(2000)

##### ขาดไป 1 เดือน (11 แถว)

###### OD

In [ ]:
doc(176970)

###### OH

In [ ]:
doc(177000)

###### PI

In [ ]:
doc(178000)

###### PH / MY

In [ ]:
doc(179000)

##### ขาดมากกว่า 2 เดือน (น้อยกว่า 11 แถว)

###### PC

In [ ]:
doc(180000)

###### MG

In [ ]:
doc(181000)

###### MY / MK / PA

In [ ]:
doc(182000)

###### PD

In [ ]:
doc(185000)

###### PH / MY

In [ ]:
doc(190000)

###### PZ

In [ ]:
doc(199000)

###### PF

In [ ]:
doc(199100)

###### PE

In [ ]:
doc(199200)

In [ ]:
# Save
payment.to_csv(
    '/content/drive/MyDrive/PEA-intern/CreditScore/df/payment.csv',
    index=False
    )

## Merging

## Recheck

In [ ]:
customer.drop_duplicates(subset='ca',inplace=True)
customer.reset_index(drop=True,inplace=True)
customer

In [ ]:
payment

In [ ]:
customer.info()

In [ ]:
customer.tsic = customer.tsic.astype('category')
customer.info()

In [ ]:
payment.info()

In [ ]:
def astype(col,type,df):
    df[col] = df[col].astype(type)
    return df.info()

In [ ]:
col = ['ca','bp','tsic','act_class','ratecat']
astype(col,str,customer)

In [ ]:
col = ['ca','bp','tsic','act_class','ratecat']
astype(col,str,payment)

## merge

In [ ]:
col = ['ca','bp','tsic','act_class','ratecat','trsg']
merge = pd.merge(payment,customer,on=col)
merge

### inner

In [ ]:
col = ['ca','bp','tsic','act_class','ratecat','trsg']
inner = customer.merge(payment,how='inner',on=col)
inner

### Left

In [ ]:
col = ['ca','ca_name','bp','tsic','act_class','ratecat','trsg']
left = customer.merge(payment,how='left',on=col)
left

In [ ]:
left.info()

In [ ]:
left['ca'] = left['ca'].str.lstrip('0')

In [ ]:
# Save
left.to_csv(
    '/content/drive/MyDrive/PEA-intern/CreditScore/df/left.csv',
    index=False
    )

### Right

In [ ]:
col = ['ca','ca_name','bp','tsic','act_class','ratecat','trsg']
right = customer.merge(payment,how='right',on=col)
right

In [ ]:
right.info()

In [ ]:
astype('bp_type',str,right)

In [ ]:
right['ca'] = right['ca'].str.lstrip('0')

In [ ]:
# Save
right.to_csv(
    '/content/drive/MyDrive/PEA-intern/CreditScore/df/left.csv',
    index=False
    )

### Merging Left & Right

#### test

In [ ]:
left[(left.ca=='20000020082')]

In [ ]:
right[(right.ca=='20000020082')]

In [ ]:
cols = left.columns
r =right[(right.ca=='20000020082')]
l = left[(left.ca=='20000020082')]

In [ ]:
for i in range(len(r)):
    for col in cols:
        r.iloc[i, right.columns.get_loc(col)] = l.iloc[0, left.columns.get_loc(col)]

r

In [ ]:
cols = ['bp_type', 'act_class', 'tsic', 'trsg','mi_date', 'mo_date', 'kva',
       'volt_buy', 'portion', 'meter_type', 'meter_size', 'ratecat', 'region',
       'sec_dep_cash', 'sec_dep_noncash', 'meter_age'
       ]

left_ca = left['ca'].unique()

right.loc[right['ca'] == left_ca[80], cols] = left.loc[left['ca'] == left_ca[80],cols].iloc[0]
right.loc[right['ca'] == left_ca[80]]

In [ ]:
right[right['ca'] == left_ca[80]]

In [ ]:
left.loc[left['ca'] == left_ca[80]]

#### Function

In [ ]:
def replace_iloc(ca):

  r = right[(right['ca'] == ca)]
  l = left[(left['ca'] == ca)]

  cols = ['bp_type', 'act_class', 'tsic', 'mi_date', 'mo_date', 'kva',
          'volt_buy', 'portion', 'meter_type', 'meter_size', 'ratecat',
          'region','sec_dep_cash', 'sec_dep_noncash', 'meter_age'
          ]

  for i in range(len(r)):
    for col in cols:
        r.iloc[i, right.columns.get_loc(col)] = l.iloc[0, left.columns.get_loc(col)]

  right[(right['ca'] == ca)] = r

  return r

#### Call Function

##### Pre Loop

In [ ]:
replace_iloc('20000022602')

In [ ]:
right[right.ca=='20000022602']

##### Loop

In [ ]:
# from tqdm import tqdm

# left_ca = left['ca'].unique().tolist()

# index = 0
# remaining_ca = left_ca[index:]
# total_ca = len(remaining_ca)

# pbar = tqdm(total=total_ca, desc="Processing", unit="ca")

# for i in left_ca:
#   replace_iloc(i)
#   pbar.update(1)

# pbar.close()

# print("All Done!")

# right.to_csv('/content/drive/MyDrive/PEA-intern/02-CreditScore_Customers/file/DataFrame/credit_customer.csv')
# print("Saved!")

##### Gem Code

In [ ]:
def replace_iloc(ca, right_dict, left_dict, cols):
    if ca in right_dict and ca in left_dict:
        right_dict[ca].loc[:, cols] = left_dict[ca].loc[left_dict[ca].index[0], cols].values

In [ ]:
common_ca = set(left['ca']).intersection(set(right['ca']))
right_dict = {ca: df for ca, df in right.groupby('ca')}
left_dict = {ca: df for ca, df in left.groupby('ca')}

In [ ]:
from tqdm import tqdm

cols = ['bp_type', 'act_class', 'tsic', 'mi_date', 'mo_date', 'kva',
        'volt_buy', 'portion', 'meter_type', 'meter_size', 'ratecat',
        'region','sec_dep_cash', 'sec_dep_noncash', 'meter_age'
        ]

pbar = tqdm(total=len(common_ca), desc="Processing", unit="ca")
for ca in common_ca:
    replace_iloc(ca, right_dict, left_dict, cols)
    pbar.update(1)
pbar.close()

right_updated = pd.concat(right_dict.values())

print("All Done!")

In [ ]:
right_updated

In [ ]:
right_updated.info()

In [ ]:
# Save
right_updated.to_csv(
    '/content/drive/MyDrive/PEA-intern/CreditScore/df/credit&payment.csv',
    index=False
    )

# P'Tuck Dataset

## 1. ts file

### Data

In [4]:
ts = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/PEA-intern/CreditScore/File/ts_12bill_pay_ready_010924.csv'))
ts

<ipython-input-4-d158038cdeaf>:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  ts = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/PEA-intern/CreditScore/File/ts_12bill_pay_ready_010924.csv'))


,ca,period,trsg,installation,amt_invoice,free,discount_operand,min_waived,tmp_dsc,OPBEL,...,TAX_ID,BRANCH,DEPT_DOC_TYPE,PAY_DOC_TYPE,PAYMENT_CH,BILL_PERIOD_F,BILL_PERIOD_D,BILL_DUEDATE_F,BILL_PAYDATE_F,partition_index
0,20000020082,202301,G18101,6001131344,22227.67,NaN,NaN,NaN,NaN,3.392029e+11,...,3140600319095,NaN,IN,PG,รับชำระเงิน กฟข ก1,2023-01-01,202301.0,2023-02-17,2023-02-14,0
1,20000020082,202308,G18101,6001131344,29752.38,NaN,NaN,NaN,NaN,3.038788e+11,...,3140600319095,NaN,IN,PG,รับชำระเงิน กฟข ก1,2023-08-01,202308.0,2023-09-18,2023-09-14,0
2,20000021030,202304,G21101,6000489476,9785.89,NaN,NaN,NaN,NaN,3.716285e+11,...,0994000159056,00067,IN,OG,รับชำระกลุ่ม กฟข ก1,2023-04-01,202304.0,2023-05-19,2023-05-29,0
3,20000021030,202307,G21101,6000489476,5860.76,NaN,NaN,NaN,NaN,3.704037e+11,...,0994000159056,00067,IN,OG,รับชำระกลุ่ม กฟข ก1,2023-07-01,202307.0,2023-08-21,2023-08-25,0
4,20000021041,202301,G21101,6000489484,2220.36,NaN,NaN,NaN,NaN,3.163781e+11,...,0994000159056,00067,IN,OG,รับชำระกลุ่ม กฟข ก1,2023-01-01,202301.0,2023-02-20,2023-02-24,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002111,20026307616,202304,B09101,6022618267,177852.70,NaN,NaN,NaN,NaN,3.715034e+11,...,0635565000470,00000,IN,PB,รับชำระเงิน กฟข น2,2023-04-01,202304.0,2023-05-22,2023-05-22,7
2002112,20026307616,202305,B09101,6022618267,167041.70,NaN,NaN,NaN,NaN,3.812538e+11,...,0635565000470,00000,IN,PB,รับชำระเงิน กฟข น2,2023-05-01,202305.0,2023-06-20,2023-06-22,7
2002113,20026311196,202307,K06101,6024860425,9493.97,NaN,NaN,NaN,NaN,3.750037e+11,...,0835562000841,00000,IN,PK,รับชำระเงิน กฟข ต2,2023-07-01,202307.0,2023-08-18,2023-09-08,7
2002114,20026318734,202304,E01101,6024863919,13739.46,NaN,NaN,NaN,NaN,3.589031e+11,...,X125485465212,00000,IN,PE,รับชำระเงิน กฟข ฉ2,2023-04-01,202304.0,2023-05-18,2023-06-07,7


In [5]:
ts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2002116 entries, 0 to 2002115
Data columns (total 29 columns):
 #   Column            Dtype  
---  ------            -----  
 0   ca                int64  
 1   period            int64  
 2   trsg              object 
 3   installation      int64  
 4   amt_invoice       float64
 5   free              object 
 6   discount_operand  object 
 7   min_waived        object 
 8   tmp_dsc           object 
 9   OPBEL             float64
 10  XBLNR             float64
 11  CA_NAME           object 
 12  KTOKL             object 
 13  TARIFTYP          object 
 14  TSIC_ID           float64
 15  AMT_DEBT          float64
 16  INSTALLMENT       object 
 17  INST_PERIOD       float64
 18  BP                float64
 19  TAX_ID            object 
 20  BRANCH            object 
 21  DEPT_DOC_TYPE     object 
 22  PAY_DOC_TYPE      object 
 23  PAYMENT_CH        object 
 24  BILL_PERIOD_F     object 
 25  BILL_PERIOD_D     float64
 26  BILL_DUEDATE_F

In [6]:
ts_profile = pd.DataFrame(pd.read_excel('/content/drive/MyDrive/PEA-intern/CreditScore/File/ts_12bill_pay_ready_dc_010924.xlsx'))
ts_profile

,summary,ca,period,trsg,installation,amt_invoice,free,discount_operand,min_waived,tmp_dsc,...,BP,TAX_ID,BRANCH,DEPT_DOC_TYPE,PAY_DOC_TYPE,PAYMENT_CH,BILL_PERIOD_F,BILL_PERIOD_D,BILL_DUEDATE_F,BILL_PAYDATE_F
0,count,2.002116e+06,2.002116e+06,2002116,2.002116e+06,2.002116e+06,7,14091,146,13595,...,1.983958e+06,1754949,1431428,1983958,1983958,1983958,1983958,1.983958e+06,1983958,1983958
1,mean,2.001467e+10,2.023065e+05,NaN,6.012864e+09,1.860460e+05,NaN,NaN,NaN,NaN,...,1.010864e+09,1376770785254.810059,123879543.377719,NaN,NaN,NaN,NaN,2.023065e+05,NaN,NaN
2,stddev,8.337617e+06,3.452053e+00,NaN,8.028278e+06,1.574007e+06,NaN,NaN,NaN,NaN,...,7.371940e+06,1799842202691.860107,18389594730.634998,NaN,NaN,NaN,NaN,3.449030e+00,NaN,NaN
3,min,2.000002e+10,2.023010e+05,A00000,6.000000e+09,0.000000e+00,X,BOI-CUST,X,X,...,1.000000e+09,.,-,IN,AM,GFMIS - ใบกำกับฯ,2023-01-01 00:00:00,2.023010e+05,2023-01-17 00:00:00,2023-01-17 00:00:00
4,max,2.002633e+10,2.023120e+05,L13401,6.025183e+09,3.062063e+08,X,VET-CUST,X,X,...,1.021976e+09,x100035798412,xxxxx,IN,ZZ,ไฟฟรี 90/50 หน่วย,2023-12-01 00:00:00,2.023120e+05,2026-09-10 00:00:00,2024-06-29 00:00:00
5,NaN,0.000000e+00,0.000000e+00,0,0.000000e+00,0.000000e+00,2002109,1988025,2001970,1988521,...,1.815800e+04,247167,570688,18158,18158,18158,18158,1.815800e+04,18158,18158


### ca

In [7]:
ts.ca.value_counts()

,count
ca,
20000020082,12
20023071341,12
20023070708,12
20023070769,12
20023070817,12
...,...
20015165177,12
20015165447,12
20015165472,12


In [8]:
ts.sort_values(by=['ca','period'],inplace=True)
ts.reset_index(drop=True,inplace=True)
ts

,ca,period,trsg,installation,amt_invoice,free,discount_operand,min_waived,tmp_dsc,OPBEL,...,TAX_ID,BRANCH,DEPT_DOC_TYPE,PAY_DOC_TYPE,PAYMENT_CH,BILL_PERIOD_F,BILL_PERIOD_D,BILL_DUEDATE_F,BILL_PAYDATE_F,partition_index
0,20000020082,202301,G18101,6001131344,22227.67,NaN,NaN,NaN,NaN,3.392029e+11,...,3140600319095,NaN,IN,PG,รับชำระเงิน กฟข ก1,2023-01-01,202301.0,2023-02-17,2023-02-14,0
1,20000020082,202302,G18101,6001131344,25754.95,NaN,NaN,NaN,NaN,3.862536e+11,...,3140600319095,NaN,IN,PG,รับชำระเงิน กฟข ก1,2023-02-01,202302.0,2023-03-17,2023-03-14,3
2,20000020082,202303,G18101,6001131344,30759.70,NaN,NaN,NaN,NaN,3.020032e+11,...,3140600319095,NaN,IN,PG,รับชำระเงิน กฟข ก1,2023-03-01,202303.0,2023-04-18,2023-04-10,3
3,20000020082,202304,G18101,6001131344,31664.86,NaN,NaN,NaN,NaN,3.192029e+11,...,3140600319095,NaN,IN,PG,รับชำระเงิน กฟข ก1,2023-04-01,202304.0,2023-05-18,2023-05-18,7
4,20000020082,202305,G18101,6001131344,31694.34,NaN,NaN,NaN,NaN,3.228781e+11,...,3140600319095,NaN,IN,PG,รับชำระเงิน กฟข ก1,2023-05-01,202305.0,2023-06-19,2023-06-16,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002111,20026328304,202308,B06101,6020082110,334.10,NaN,NaN,NaN,NaN,3.727535e+11,...,0206101818752,00001,IN,MG,GFMIS - ใบกำกับฯ,2023-08-01,202308.0,2023-09-21,2023-11-03,6
2002112,20026328304,202309,B06101,6020082110,334.10,NaN,NaN,NaN,NaN,3.668787e+11,...,0206101818752,00001,IN,MG,GFMIS - ใบกำกับฯ,2023-09-01,202309.0,2023-10-24,2023-12-04,5
2002113,20026328304,202310,B06101,6020082110,334.10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
2002114,20026328304,202311,B06101,6020082110,334.10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [9]:
ts.ca.nunique()

166843

### free

In [10]:
ts.free.value_counts()

,count
free,
X,7


In [11]:
ts[ts.free == 'X']

,ca,period,trsg,installation,amt_invoice,free,discount_operand,min_waived,tmp_dsc,OPBEL,...,TAX_ID,BRANCH,DEPT_DOC_TYPE,PAY_DOC_TYPE,PAYMENT_CH,BILL_PERIOD_F,BILL_PERIOD_D,BILL_DUEDATE_F,BILL_PAYDATE_F,partition_index
796272,20013738857,202301,F09101,6011274933,47.59,X,NaN,NaN,NaN,3.332530e+11,...,3300200119071,NaN,IN,MA,ไฟฟรี 90/50 หน่วย,2023-01-01,202301.0,2023-01-24,2023-01-18,0
796273,20013738857,202302,F09101,6011274933,117.54,X,NaN,NaN,NaN,3.805037e+11,...,3300200119071,NaN,IN,MA,ไฟฟรี 90/50 หน่วย,2023-02-01,202302.0,2023-02-21,2023-02-15,5
1841892,20024788860,202301,E14103,6023599996,0.00,X,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1841893,20024788860,202302,E14103,6023599996,0.00,X,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1841894,20024788860,202303,E14103,6023599996,0.00,X,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1841895,20024788860,202304,E14103,6023599996,0.00,X,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1841896,20024788860,202305,E14103,6023599996,0.00,X,NaN,NaN,NaN,3.002064e+11,...,1342100090874,NaN,IN,PE,รับชำระเงิน กฟข ฉ2,2023-05-01,202305.0,2023-07-03,2023-07-04,6


In [12]:
ts[ts.ca==20013738857]

,ca,period,trsg,installation,amt_invoice,free,discount_operand,min_waived,tmp_dsc,OPBEL,...,TAX_ID,BRANCH,DEPT_DOC_TYPE,PAY_DOC_TYPE,PAYMENT_CH,BILL_PERIOD_F,BILL_PERIOD_D,BILL_DUEDATE_F,BILL_PAYDATE_F,partition_index
796272,20013738857,202301,F09101,6011274933,47.59,X,NaN,NaN,NaN,3.332530e+11,...,3300200119071,NaN,IN,MA,ไฟฟรี 90/50 หน่วย,2023-01-01,202301.0,2023-01-24,2023-01-18,0
796273,20013738857,202302,F09101,6011274933,117.54,X,NaN,NaN,NaN,3.805037e+11,...,3300200119071,NaN,IN,MA,ไฟฟรี 90/50 หน่วย,2023-02-01,202302.0,2023-02-21,2023-02-15,5
796274,20013738857,202303,F09101,6011274933,494.41,NaN,NaN,NaN,NaN,3.061279e+11,...,3300200119071,NaN,IN,MY,MPAYMENT-ธนาคารกสิกรไทย จำกัด (มหาชน) App,2023-03-01,202303.0,2023-03-24,2023-03-19,7
796275,20013738857,202304,F09101,6011274933,430.53,NaN,NaN,NaN,NaN,3.186280e+11,...,3300200119071,NaN,IN,PF,รับชำระเงิน กฟข ฉ3,2023-04-01,202304.0,2023-04-23,2023-04-18,0
796276,20013738857,202305,F09101,6011274933,0.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
796277,20013738857,202306,F09101,6011274933,18.89,NaN,NaN,NaN,NaN,3.077535e+11,...,3300200119071,NaN,IN,PF,รับชำระเงิน กฟข ฉ3,2023-06-01,202306.0,2023-06-23,2023-07-03,6
796278,20013738857,202307,F09101,6011274933,284.91,NaN,NaN,NaN,NaN,3.010066e+11,...,3300200119071,NaN,IN,MY,MPAYMENT-ธนาคารออมสิน(App.),2023-07-01,202307.0,2023-07-24,2023-07-23,1
796279,20013738857,202308,F09101,6011274933,40294.36,NaN,NaN,NaN,NaN,3.002067e+11,...,3300200119071,NaN,IN,PF,รับชำระเงิน กฟข ฉ3,2023-08-01,202308.0,2023-09-04,2023-08-22,7
796280,20013738857,202309,F09101,6011274933,33101.45,NaN,NaN,NaN,NaN,3.206282e+11,...,3300200119071,NaN,IN,MY,MPAYMENT-ธนาคารออมสิน(App.),2023-09-01,202309.0,2023-10-02,2023-09-19,4
796281,20013738857,202310,F09101,6011274933,29329.20,NaN,NaN,NaN,NaN,3.923788e+11,...,3300200119071,NaN,IN,PF,รับชำระเงิน กฟข ฉ3,2023-10-01,202310.0,2023-11-06,2023-11-10,2


In [13]:
# new customer ?
ts[ts.ca==20024788860]

,ca,period,trsg,installation,amt_invoice,free,discount_operand,min_waived,tmp_dsc,OPBEL,...,TAX_ID,BRANCH,DEPT_DOC_TYPE,PAY_DOC_TYPE,PAYMENT_CH,BILL_PERIOD_F,BILL_PERIOD_D,BILL_DUEDATE_F,BILL_PAYDATE_F,partition_index
1841892,20024788860,202301,E14103,6023599996,0.00,X,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1841893,20024788860,202302,E14103,6023599996,0.00,X,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1841894,20024788860,202303,E14103,6023599996,0.00,X,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1841895,20024788860,202304,E14103,6023599996,0.00,X,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1841896,20024788860,202305,E14103,6023599996,0.00,X,NaN,NaN,NaN,3.002064e+11,...,1342100090874,NaN,IN,PE,รับชำระเงิน กฟข ฉ2,2023-05-01,202305.0,2023-07-03,2023-07-04,6
1841897,20024788860,202306,E14103,6023599996,3519.12,NaN,NaN,NaN,NaN,3.310036e+11,...,1342100090874,NaN,IN,PE,รับชำระเงิน กฟข ฉ2,2023-06-01,202306.0,2023-07-17,2023-07-04,2
1841898,20024788860,202307,E14103,6023599996,6967.20,NaN,NaN,NaN,NaN,3.081284e+11,...,1342100090874,NaN,IN,PE,รับชำระเงิน กฟข ฉ2,2023-07-01,202307.0,2023-08-17,2023-08-17,3
1841899,20024788860,202308,E14103,6023599996,3867.29,NaN,NaN,NaN,NaN,3.907538e+11,...,1342100090874,NaN,IN,PE,รับชำระเงิน กฟข ฉ2,2023-08-01,202308.0,2023-09-18,2023-09-19,0
1841900,20024788860,202309,E14103,6023599996,3560.28,NaN,NaN,NaN,NaN,3.442032e+11,...,1342100090874,NaN,IN,PE,รับชำระเงิน กฟข ฉ2,2023-09-01,202309.0,2023-10-24,2023-10-24,5
1841901,20024788860,202310,E14103,6023599996,4913.70,NaN,NaN,NaN,NaN,3.110037e+11,...,1342100090874,NaN,IN,PE,รับชำระเงิน กฟข ฉ2,2023-10-01,202310.0,2023-11-17,2023-11-17,4


### discount_operand

In [14]:
ts.discount_operand.value_counts()

,count
discount_operand,
OTG-OMS,13694
BOI-CUST,282
DRUG-D-QTY,60
VET-CUST,43
HOME-D-FLG,12


In [15]:
ts[ts.discount_operand == 'OTG-OMS']

,ca,period,trsg,installation,amt_invoice,free,discount_operand,min_waived,tmp_dsc,OPBEL,...,TAX_ID,BRANCH,DEPT_DOC_TYPE,PAY_DOC_TYPE,PAYMENT_CH,BILL_PERIOD_F,BILL_PERIOD_D,BILL_DUEDATE_F,BILL_PAYDATE_F,partition_index
1221,20000240259,202310,G05301,6001278067,704360.87,NaN,OTG-OMS,NaN,NaN,3.349033e+11,...,0265546000030,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-10-01,202310.0,2023-11-20,2023-11-16,6
1243,20000241686,202308,G05301,6001267344,77859.52,NaN,OTG-OMS,NaN,NaN,3.514039e+11,...,0133555005285,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-08-01,202308.0,2023-09-18,2023-09-25,4
1635,20000259266,202304,G06301,6001119347,30302.41,NaN,OTG-OMS,NaN,NaN,3.149029e+11,...,0145542000036,00000,IN,PG,รับชำระเงิน กฟข ก1,2023-04-01,202304.0,2023-05-18,2023-05-16,3
1636,20000259266,202305,G06301,6001119347,29842.58,NaN,OTG-OMS,NaN,NaN,3.188782e+11,...,0145542000036,00000,IN,PG,รับชำระเงิน กฟข ก1,2023-05-01,202305.0,2023-06-19,2023-06-19,7
1637,20000259266,202306,G06301,6001119347,28411.74,NaN,OTG-OMS,NaN,NaN,3.325037e+11,...,0145542000036,00000,IN,PG,รับชำระเงิน กฟข ก1,2023-06-01,202306.0,2023-07-24,2023-07-14,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999313,20026233781,202306,J10101,6004220415,82428.24,NaN,OTG-OMS,NaN,NaN,3.110035e+11,...,0765561000172,00000,IN,PJ,รับชำระเงิน กฟข ต1,2023-06-01,202306.0,2023-07-17,2023-07-19,6
1999348,20026233824,202305,E03101,6015666566,5922.73,NaN,OTG-OMS,NaN,NaN,3.995035e+11,...,0353532000056,00000,IN,PE,รับชำระเงิน กฟข ฉ2,2023-05-01,202305.0,2023-06-19,2023-06-21,6
1999351,20026233824,202308,E03101,6015666566,4328.92,NaN,OTG-OMS,NaN,NaN,3.685034e+11,...,0353532000056,00000,IN,PE,รับชำระเงิน กฟข ฉ2,2023-08-01,202308.0,2023-09-18,2023-09-22,3
1999352,20026233824,202309,E03101,6015666566,3551.48,NaN,OTG-OMS,NaN,NaN,3.845040e+11,...,0353532000056,00000,IN,PE,รับชำระเงิน กฟข ฉ2,2023-09-01,202309.0,2023-10-24,2023-10-24,0


In [16]:
ts[ts.ca==20000240259]

,ca,period,trsg,installation,amt_invoice,free,discount_operand,min_waived,tmp_dsc,OPBEL,...,TAX_ID,BRANCH,DEPT_DOC_TYPE,PAY_DOC_TYPE,PAYMENT_CH,BILL_PERIOD_F,BILL_PERIOD_D,BILL_DUEDATE_F,BILL_PAYDATE_F,partition_index
1212,20000240259,202301,G05301,6001278067,998661.20,NaN,NaN,NaN,NaN,3.988786e+11,...,0265546000030,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-01-01,202301.0,2023-02-20,2023-02-20,7
1213,20000240259,202302,G05301,6001278067,976896.66,NaN,NaN,NaN,NaN,3.235036e+11,...,0265546000030,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-02-01,202302.0,2023-03-20,2023-03-20,7
1214,20000240259,202303,G05301,6001278067,1055962.28,NaN,NaN,NaN,NaN,3.085035e+11,...,0265546000030,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-03-01,202303.0,2023-04-20,2023-04-20,1
1215,20000240259,202304,G05301,6001278067,992160.81,NaN,NaN,NaN,NaN,3.080035e+11,...,0265546000030,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-04-01,202304.0,2023-05-22,2023-05-18,1
1216,20000240259,202305,G05301,6001278067,921022.47,NaN,NaN,NaN,NaN,3.010064e+11,...,0265546000030,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-05-01,202305.0,2023-06-20,2023-06-21,1
1217,20000240259,202306,G05301,6001278067,923096.60,NaN,NaN,NaN,NaN,3.016065e+11,...,0265546000030,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-06-01,202306.0,2023-07-20,2023-07-20,6
1218,20000240259,202307,G05301,6001278067,856780.22,NaN,NaN,NaN,NaN,3.890036e+11,...,0265546000030,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-07-01,202307.0,2023-08-21,2023-08-18,0
1219,20000240259,202308,G05301,6001278067,853468.52,NaN,NaN,NaN,NaN,3.948788e+11,...,0265546000030,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-08-01,202308.0,2023-09-20,2023-09-20,6
1220,20000240259,202309,G05301,6001278067,734825.45,NaN,NaN,NaN,NaN,3.991284e+11,...,0265546000030,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-09-01,202309.0,2023-10-24,2023-10-26,3
1221,20000240259,202310,G05301,6001278067,704360.87,NaN,OTG-OMS,NaN,NaN,3.349033e+11,...,0265546000030,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-10-01,202310.0,2023-11-20,2023-11-16,6


In [17]:
ts[ts.discount_operand=='OTG-OMS'].iloc[0]

,1221
ca,20000240259
period,202310
trsg,G05301
installation,6001278067
amt_invoice,704360.87
free,NaN
discount_operand,OTG-OMS
min_waived,NaN
tmp_dsc,NaN
OPBEL,334903313198.0


In [18]:
ts[ts.ca==20000241686]

,ca,period,trsg,installation,amt_invoice,free,discount_operand,min_waived,tmp_dsc,OPBEL,...,TAX_ID,BRANCH,DEPT_DOC_TYPE,PAY_DOC_TYPE,PAYMENT_CH,BILL_PERIOD_F,BILL_PERIOD_D,BILL_DUEDATE_F,BILL_PAYDATE_F,partition_index
1236,20000241686,202301,G05301,6001267344,95380.57,NaN,NaN,NaN,NaN,3.402031e+11,...,0133555005285,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-01-01,202301.0,2023-02-17,2023-02-28,5
1237,20000241686,202302,G05301,6001267344,101574.82,NaN,NaN,NaN,NaN,3.872535e+11,...,0133555005285,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-02-01,202302.0,2023-03-17,2023-03-27,5
1238,20000241686,202303,G05301,6001267344,118952.91,NaN,NaN,NaN,NaN,3.040035e+11,...,0133555005285,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-03-01,202303.0,2023-04-18,2023-04-24,2
1239,20000241686,202304,G05301,6001267344,123812.70,NaN,NaN,NaN,NaN,3.159031e+11,...,0133555005285,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-04-01,202304.0,2023-05-18,2023-05-29,6
1240,20000241686,202305,G05301,6001267344,110283.72,NaN,NaN,NaN,NaN,3.163783e+11,...,0133555005285,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-05-01,202305.0,2023-06-19,2023-06-26,7
1241,20000241686,202306,G05301,6001267344,86992.12,NaN,NaN,NaN,NaN,3.893784e+11,...,0133555005285,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-06-01,202306.0,2023-07-17,2023-07-25,1
1242,20000241686,202307,G05301,6001267344,99096.84,NaN,NaN,NaN,NaN,3.715035e+11,...,0133555005285,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-07-01,202307.0,2023-08-17,2023-08-25,5
1243,20000241686,202308,G05301,6001267344,77859.52,NaN,OTG-OMS,NaN,NaN,3.514039e+11,...,0133555005285,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-08-01,202308.0,2023-09-18,2023-09-25,4
1244,20000241686,202309,G05301,6001267344,94314.51,NaN,NaN,NaN,NaN,3.991284e+11,...,0133555005285,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-09-01,202309.0,2023-10-24,2023-10-24,3
1245,20000241686,202310,G05301,6001267344,89100.72,NaN,NaN,NaN,NaN,3.720037e+11,...,0133555005285,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-10-01,202310.0,2023-11-17,2023-11-29,6


### min_waived

In [19]:
ts.min_waived.value_counts()

,count
min_waived,
X,146


In [20]:
ts[ts.min_waived == 'X']

,ca,period,trsg,installation,amt_invoice,free,discount_operand,min_waived,tmp_dsc,OPBEL,...,TAX_ID,BRANCH,DEPT_DOC_TYPE,PAY_DOC_TYPE,PAYMENT_CH,BILL_PERIOD_F,BILL_PERIOD_D,BILL_DUEDATE_F,BILL_PAYDATE_F,partition_index
61498,20000978130,202311,G16101,6001116773,1385204.66,NaN,NaN,X,NaN,3.882539e+11,...,0105547117179,00001,IN,PI,รับชำระเงิน กฟข ก3,2023-11-01,202311.0,2023-12-20,2023-12-18,6
61499,20000978130,202312,G16101,6001116773,970219.49,NaN,NaN,X,NaN,3.980040e+11,...,0105547117179,00001,IN,PI,รับชำระเงิน กฟข ก3,2023-12-01,202312.0,2024-01-22,2024-01-18,2
90867,20001110553,202304,G16101,6001119012,236606.09,NaN,NaN,X,NaN,3.055033e+11,...,0105534002327,00003,IN,MD,ชำระเงินแบบหักบัญชี,2023-04-01,202304.0,2023-05-19,2023-05-19,7
260468,20002784922,202309,I01101,6003514862,623553.77,NaN,NaN,X,NaN,3.316282e+11,...,0735547000527,00000,IN,PI,รับชำระเงิน กฟข ก3,2023-09-01,202309.0,2023-10-24,2023-10-24,5
271957,20002892750,202302,I03305,6003196763,7114.94,NaN,NaN,X,NaN,3.529034e+11,...,3710900701411,NaN,IN,PI,รับชำระเงิน กฟข ก3,2023-02-01,202302.0,2023-03-20,2023-03-24,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1929839,20025590097,202312,J11102,6024323346,20491.02,NaN,NaN,X,NaN,3.355038e+11,...,3340400057737,NaN,IN,PJ,รับชำระเงิน กฟข ต1,2023-12-01,202312.0,2024-01-22,2024-01-18,2
1937684,20025668110,202309,J11102,6024473496,276981.71,NaN,NaN,X,NaN,3.018083e+11,...,3629900020021,NaN,IN,PJ,รับชำระเงิน กฟข ต1,2023-09-01,202309.0,2023-10-30,2023-10-18,7
1956747,20025849226,202304,J09106,6022684289,15045.32,NaN,NaN,X,NaN,3.545033e+11,...,0703565000906,00001,IN,MY,PPAYMENT-ธ.กรุงเทพ จำกัด(มหาชน) (PEA App),2023-04-01,202304.0,2023-05-22,2023-05-21,7
1956759,20025849228,202304,J09106,6019232948,9959.93,NaN,NaN,X,NaN,3.545033e+11,...,0703565000906,00000,IN,MY,PPAYMENT-ธ.กรุงเทพ จำกัด(มหาชน) (PEA App),2023-04-01,202304.0,2023-05-22,2023-05-21,0


In [21]:
ts[ts.ca==20000978130]

,ca,period,trsg,installation,amt_invoice,free,discount_operand,min_waived,tmp_dsc,OPBEL,...,TAX_ID,BRANCH,DEPT_DOC_TYPE,PAY_DOC_TYPE,PAYMENT_CH,BILL_PERIOD_F,BILL_PERIOD_D,BILL_DUEDATE_F,BILL_PAYDATE_F,partition_index
61488,20000978130,202301,G16101,6001116773,2735228.47,NaN,NaN,NaN,NaN,3.484032e+11,...,0105547117179,00001,IN,PI,รับชำระเงิน กฟข ก3,2023-01-01,202301.0,2023-02-20,2023-02-20,4
61489,20000978130,202302,G16101,6001116773,1259137.12,NaN,NaN,NaN,NaN,3.225035e+11,...,0105547117179,00001,IN,PI,รับชำระเงิน กฟข ก3,2023-02-01,202302.0,2023-03-20,2023-03-20,0
61490,20000978130,202303,G16101,6001116773,1471404.86,NaN,NaN,NaN,NaN,3.095035e+11,...,0105547117179,00001,IN,PI,รับชำระเงิน กฟข ก3,2023-03-01,202303.0,2023-04-20,2023-04-18,4
61491,20000978130,202304,G16101,6001116773,1520138.59,NaN,NaN,NaN,NaN,3.060034e+11,...,0105547117179,00001,IN,PI,รับชำระเงิน กฟข ก3,2023-04-01,202304.0,2023-05-22,2023-05-18,5
61492,20000978130,202305,G16101,6001116773,1196808.55,NaN,NaN,NaN,NaN,3.230035e+11,...,0105547117179,00001,IN,PI,รับชำระเงิน กฟข ก3,2023-05-01,202305.0,2023-06-20,2023-06-15,5
61493,20000978130,202306,G16101,6001116773,1228933.40,NaN,NaN,NaN,NaN,3.272031e+11,...,0105547117179,00001,IN,PI,รับชำระเงิน กฟข ก3,2023-06-01,202306.0,2023-07-20,2023-07-17,6
61494,20000978130,202307,G16101,6001116773,319989.28,NaN,NaN,NaN,NaN,3.895038e+11,...,0105547117179,00001,IN,PI,รับชำระเงิน กฟข ก3,2023-07-01,202307.0,2023-08-21,2023-08-17,6
61495,20000978130,202308,G16101,6001116773,2089312.42,NaN,NaN,NaN,NaN,3.262036e+11,...,0105547117179,00001,IN,PI,รับชำระเงิน กฟข ก3,2023-08-01,202308.0,2023-09-20,2023-09-18,5
61496,20000978130,202309,G16101,6001116773,650806.83,NaN,NaN,NaN,NaN,3.991284e+11,...,0105547117179,00001,IN,PI,รับชำระเงิน กฟข ก3,2023-09-01,202309.0,2023-10-24,2023-10-20,7
61497,20000978130,202310,G16101,6001116773,924739.85,NaN,NaN,NaN,NaN,3.456281e+11,...,0105547117179,00001,IN,PI,รับชำระเงิน กฟข ก3,2023-10-01,202310.0,2023-11-20,2023-11-16,5


In [22]:
pd.DataFrame((ts[ts.ca==20000978130].iloc[10], ts[ts.ca==20000978130].iloc[11],ts[ts.ca==20000978130].iloc[0])).T

,61498,61499,61488
ca,20000978130,20000978130,20000978130
period,202311,202312,202301
trsg,G16101,G16101,G16101
installation,6001116773,6001116773,6001116773
amt_invoice,1385204.66,970219.49,2735228.47
free,NaN,NaN,NaN
discount_operand,NaN,NaN,NaN
min_waived,X,X,NaN
tmp_dsc,NaN,NaN,NaN
OPBEL,388253910398.0,398004009481.0,348403225371.0


In [23]:
ts[ts.ca==20001110553]

,ca,period,trsg,installation,amt_invoice,free,discount_operand,min_waived,tmp_dsc,OPBEL,...,TAX_ID,BRANCH,DEPT_DOC_TYPE,PAY_DOC_TYPE,PAYMENT_CH,BILL_PERIOD_F,BILL_PERIOD_D,BILL_DUEDATE_F,BILL_PAYDATE_F,partition_index
90864,20001110553,202301,G16101,6001119012,3486492.13,NaN,NaN,NaN,NaN,3.487532e+11,...,0105534002327,00003,IN,MD,ชำระเงินแบบหักบัญชี,2023-01-01,202301.0,2023-02-20,2023-02-20,3
90865,20001110553,202302,G16101,6001119012,3559946.35,NaN,NaN,NaN,NaN,3.200034e+11,...,0105534002327,00003,IN,MD,ชำระเงินแบบหักบัญชี,2023-02-01,202302.0,2023-03-20,2023-03-20,6
90866,20001110553,202303,G16101,6001119012,2821676.74,NaN,NaN,NaN,NaN,3.080034e+11,...,0105534002327,00003,IN,MD,ชำระเงินแบบหักบัญชี,2023-03-01,202303.0,2023-04-19,2023-04-19,1
90867,20001110553,202304,G16101,6001119012,236606.09,NaN,NaN,X,NaN,3.055033e+11,...,0105534002327,00003,IN,MD,ชำระเงินแบบหักบัญชี,2023-04-01,202304.0,2023-05-19,2023-05-19,7
90868,20001110553,202305,G16101,6001119012,201136.83,NaN,NaN,NaN,NaN,3.190038e+11,...,0105534002327,00003,IN,MD,ชำระเงินแบบหักบัญชี,2023-05-01,202305.0,2023-06-19,2023-06-19,1
90869,20001110553,202306,G16101,6001119012,3662149.54,NaN,NaN,NaN,NaN,3.023781e+11,...,0105534002327,00003,IN,MD,ชำระเงินแบบหักบัญชี,2023-06-01,202306.0,2023-07-19,2023-07-19,1
90870,20001110553,202307,G16101,6001119012,3691024.95,NaN,NaN,NaN,NaN,3.875036e+11,...,0105534002327,00003,IN,MD,ชำระเงินแบบหักบัญชี,2023-07-01,202307.0,2023-08-21,2023-08-21,2
90871,20001110553,202308,G16101,6001119012,3984572.74,NaN,NaN,NaN,NaN,3.938785e+11,...,0105534002327,00003,IN,MD,ชำระเงินแบบหักบัญชี,2023-08-01,202308.0,2023-09-19,2023-09-19,1
90872,20001110553,202309,G16101,6001119012,3070700.32,NaN,NaN,NaN,NaN,3.986283e+11,...,0105534002327,00003,IN,MD,ชำระเงินแบบหักบัญชี,2023-09-01,202309.0,2023-10-19,2023-10-19,3
90873,20001110553,202310,G16101,6001119012,4743124.25,NaN,NaN,NaN,NaN,3.520038e+11,...,0105534002327,00003,IN,MD,ชำระเงินแบบหักบัญชี,2023-10-01,202310.0,2023-11-20,2023-11-20,7


### tmp_dsc

In [24]:
ts.tmp_dsc.value_counts()

,count
tmp_dsc,
X,13595


In [25]:
ts[ts.tmp_dsc=='X']

,ca,period,trsg,installation,amt_invoice,free,discount_operand,min_waived,tmp_dsc,OPBEL,...,TAX_ID,BRANCH,DEPT_DOC_TYPE,PAY_DOC_TYPE,PAYMENT_CH,BILL_PERIOD_F,BILL_PERIOD_D,BILL_DUEDATE_F,BILL_PAYDATE_F,partition_index
4668,20000335681,202301,G05301,6001117107,334.10,NaN,NaN,NaN,X,3.497535e+11,...,NaN,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-01-01,202301.0,2023-02-20,2023-02-21,1
4669,20000335681,202302,G05301,6001117107,334.10,NaN,NaN,NaN,X,3.205034e+11,...,NaN,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-02-01,202302.0,2023-03-20,2023-03-20,1
4670,20000335681,202303,G05301,6001117107,334.10,NaN,NaN,NaN,X,3.482034e+11,...,NaN,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-03-01,202303.0,2023-04-20,2023-04-27,0
4674,20000335681,202307,G05301,6001117107,334.10,NaN,NaN,NaN,X,3.165035e+11,...,NaN,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-07-01,202307.0,2023-08-21,2023-08-28,7
4675,20000335681,202308,G05301,6001117107,334.10,NaN,NaN,NaN,X,3.943786e+11,...,NaN,0000,IN,PG,รับชำระเงิน กฟข ก1,2023-08-01,202308.0,2023-09-20,2023-09-21,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2001404,20026279470,202309,H08201,6018751971,1608.72,NaN,NaN,NaN,X,3.845040e+11,...,0205549026234,0000,IN,PH,รับชำระเงิน กฟข ก2,2023-09-01,202309.0,2023-10-24,2023-10-25,1
2001405,20026279470,202310,H08201,6018751971,1633.38,NaN,NaN,NaN,X,3.270037e+11,...,0205549026234,0000,IN,PH,รับชำระเงิน กฟข ก2,2023-10-01,202310.0,2023-11-20,2023-11-28,3
2001406,20026279470,202311,H08201,6018751971,4633.32,NaN,NaN,NaN,X,3.997539e+11,...,0205549026234,0000,IN,PH,รับชำระเงิน กฟข ก2,2023-11-01,202311.0,2023-12-18,2023-12-18,5
2001897,20026302740,202310,I03305,6021355634,10629.27,NaN,NaN,NaN,X,3.002100e+11,...,3400600223222,NaN,IN,PI,รับชำระเงิน กฟข ก3,2023-10-01,202310.0,2023-11-20,2023-11-27,2


## 2. ca file

In [26]:
ca = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/PEA-intern/CreditScore/File/ca_status_010924.csv'))
ca

,ca,first_mi_1,last_mo_1,last_sec_dep_cash_sum,last_sec_dep_noncash_sum,sum_sec_dep,cus_age,c_status,partition_index
0,20000020082,2008-10-02,9999-12-31,64000.0,0.0,64000.0,15.0,active,0
1,20000021030,2008-05-13,9999-12-31,32000.0,0.0,32000.0,16.0,active,0
2,20000021041,2008-05-14,9999-12-31,32000.0,0.0,32000.0,16.0,active,0
3,20000021063,2008-05-13,9999-12-31,32000.0,0.0,32000.0,16.0,active,0
4,20000021083,2008-05-14,9999-12-31,32000.0,0.0,32000.0,16.0,active,0
...,...,...,...,...,...,...,...,...,...
166838,20026307616,2023-01-25,9999-12-31,320000.0,0.0,320000.0,1.0,active,0
166839,20026311196,2023-01-26,9999-12-31,64000.0,0.0,64000.0,1.0,active,0
166840,20026311209,2023-01-25,9999-12-31,0.0,0.0,0.0,1.0,active,0
166841,20026318734,2023-01-27,9999-12-31,0.0,0.0,0.0,1.0,active,0


In [27]:
ca_profile = pd.DataFrame(pd.read_excel('/content/drive/MyDrive/PEA-intern/CreditScore/File/ca_status_dc_010924.xlsx'))
ca_profile

,summary,ca,first_mi_1,last_mo_1,last_sec_dep_cash_sum,last_sec_dep_noncash_sum,sum_sec_dep,cus_age,c_status
0,count,1.668430e+05,166174,166174,1.661740e+05,1.661740e+05,1.661740e+05,166174.000000,166174
1,mean,2.001467e+10,NaN,NaN,7.013811e+04,1.712601e+05,2.413982e+05,9.728092,NaN
2,stddev,8.337640e+06,NaN,NaN,4.343732e+05,2.065105e+06,2.113542e+06,4.367579,NaN
3,min,2.000002e+10,2001-08-27 00:00:00,2023-06-01 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,active
4,max,2.002633e+10,2024-03-01 00:00:00,9999-12-31 00:00:00,7.000000e+07,3.695406e+08,3.695406e+08,23.000000,inactive
5,NaN,0.000000e+00,669,669,6.690000e+02,6.690000e+02,6.690000e+02,669.000000,669


In [28]:
ca.sort_values(by='ca',inplace=True)
ca.reset_index(drop=True,inplace=True)
ca

,ca,first_mi_1,last_mo_1,last_sec_dep_cash_sum,last_sec_dep_noncash_sum,sum_sec_dep,cus_age,c_status,partition_index
0,20000020082,2008-10-02,9999-12-31,64000.0,0.0,64000.0,15.0,active,0
1,20000021030,2008-05-13,9999-12-31,32000.0,0.0,32000.0,16.0,active,0
2,20000021041,2008-05-14,9999-12-31,32000.0,0.0,32000.0,16.0,active,0
3,20000021063,2008-05-13,9999-12-31,32000.0,0.0,32000.0,16.0,active,0
4,20000021083,2008-05-14,9999-12-31,32000.0,0.0,32000.0,16.0,active,0
...,...,...,...,...,...,...,...,...,...
166838,20026307616,2023-01-25,9999-12-31,320000.0,0.0,320000.0,1.0,active,0
166839,20026311196,2023-01-26,9999-12-31,64000.0,0.0,64000.0,1.0,active,0
166840,20026311209,2023-01-25,9999-12-31,0.0,0.0,0.0,1.0,active,0
166841,20026318734,2023-01-27,9999-12-31,0.0,0.0,0.0,1.0,active,0


## 3. merge ts and ca

In [29]:
merge = pd.merge(ca, ts, on="ca")
merge

,ca,first_mi_1,last_mo_1,last_sec_dep_cash_sum,last_sec_dep_noncash_sum,sum_sec_dep,cus_age,c_status,partition_index_x,period,...,TAX_ID,BRANCH,DEPT_DOC_TYPE,PAY_DOC_TYPE,PAYMENT_CH,BILL_PERIOD_F,BILL_PERIOD_D,BILL_DUEDATE_F,BILL_PAYDATE_F,partition_index_y
0,20000020082,2008-10-02,9999-12-31,64000.0,0.0,64000.0,15.0,active,0,202301,...,3140600319095,NaN,IN,PG,รับชำระเงิน กฟข ก1,2023-01-01,202301.0,2023-02-17,2023-02-14,0
1,20000020082,2008-10-02,9999-12-31,64000.0,0.0,64000.0,15.0,active,0,202302,...,3140600319095,NaN,IN,PG,รับชำระเงิน กฟข ก1,2023-02-01,202302.0,2023-03-17,2023-03-14,3
2,20000020082,2008-10-02,9999-12-31,64000.0,0.0,64000.0,15.0,active,0,202303,...,3140600319095,NaN,IN,PG,รับชำระเงิน กฟข ก1,2023-03-01,202303.0,2023-04-18,2023-04-10,3
3,20000020082,2008-10-02,9999-12-31,64000.0,0.0,64000.0,15.0,active,0,202304,...,3140600319095,NaN,IN,PG,รับชำระเงิน กฟข ก1,2023-04-01,202304.0,2023-05-18,2023-05-18,7
4,20000020082,2008-10-02,9999-12-31,64000.0,0.0,64000.0,15.0,active,0,202305,...,3140600319095,NaN,IN,PG,รับชำระเงิน กฟข ก1,2023-05-01,202305.0,2023-06-19,2023-06-16,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002111,20026328304,2023-01-22,9999-12-31,0.0,0.0,0.0,1.0,active,0,202308,...,0206101818752,00001,IN,MG,GFMIS - ใบกำกับฯ,2023-08-01,202308.0,2023-09-21,2023-11-03,6
2002112,20026328304,2023-01-22,9999-12-31,0.0,0.0,0.0,1.0,active,0,202309,...,0206101818752,00001,IN,MG,GFMIS - ใบกำกับฯ,2023-09-01,202309.0,2023-10-24,2023-12-04,5
2002113,20026328304,2023-01-22,9999-12-31,0.0,0.0,0.0,1.0,active,0,202310,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
2002114,20026328304,2023-01-22,9999-12-31,0.0,0.0,0.0,1.0,active,0,202311,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [30]:
merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2002116 entries, 0 to 2002115
Data columns (total 37 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   ca                        int64  
 1   first_mi_1                object 
 2   last_mo_1                 object 
 3   last_sec_dep_cash_sum     float64
 4   last_sec_dep_noncash_sum  float64
 5   sum_sec_dep               float64
 6   cus_age                   float64
 7   c_status                  object 
 8   partition_index_x         int64  
 9   period                    int64  
 10  trsg                      object 
 11  installation              int64  
 12  amt_invoice               float64
 13  free                      object 
 14  discount_operand          object 
 15  min_waived                object 
 16  tmp_dsc                   object 
 17  OPBEL                     float64
 18  XBLNR                     float64
 19  CA_NAME                   object 
 20  KTOKL                   

In [31]:
merge.ca.value_counts()

,count
ca,
20000020082,12
20020435479,12
20020433513,12
20020433533,12
20020433535,12
...,...
20009581014,12
20009581041,12
20009581047,12


In [32]:
merge.to_csv(
    '/content/drive/MyDrive/PEA-intern/CreditScore/credit_customers01.csv',
    index=False
)

## 4. customer profile

### Data

In [33]:
profile = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/PEA-intern/CreditScore/My data/data/customer_profile.csv'))
profile

,trsg,period,ca,installation,ratecat,volt_sell,pea_meter_number,kwh_pk,kwh_pp_op,kwh_op_h,kwh_total,amt_kwh_pk,amt_kwh_op,amt_kwh_tot
0,A01101,202307,20024376685,6023273775,2214,4,6300570830,4224.94,2902.41,2968.51,10095.86,21604.22,15286.11,36890.33
1,A01101,202307,20024397946,6008298139,2114,4,6500655831,1475.31,45.30,62.10,1607.12,0.00,0.00,6281.58
2,A01101,202307,20024424105,6023476266,3224,4,6300578985,7323.60,2027.76,4309.30,13660.66,30641.21,16499.80,47141.01
3,A01101,202307,20024475793,6006434186,2114,4,8740030,1040.00,0.00,0.00,1040.00,0.00,0.00,4064.95
4,A01101,202307,20024450582,6023410832,5124,4,6300570944,3586.83,2740.74,2716.77,9044.34,15006.94,14209.72,29216.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2156775,L12201,202312,20023507113,6022570496,2114,4,6500666850,1950.40,1726.40,1509.60,5186.40,0.00,0.00,20271.57
2156776,L12201,202312,20023653988,6022693484,3224,4,6500666856,3959.23,2587.54,3526.75,10073.52,16565.02,15919.78,32484.80
2156777,L12201,202312,20024083098,6023338751,4212,2,6300033427,3159000.00,2659800.00,3156000.00,8974800.00,12959797.50,15033261.42,27993058.92
2156778,L12201,202312,20024083098,6024569020,4224,4,25244792,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [34]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2156780 entries, 0 to 2156779
Data columns (total 14 columns):
 #   Column            Dtype  
---  ------            -----  
 0   trsg              object 
 1   period            int64  
 2   ca                int64  
 3   installation      int64  
 4   ratecat           object 
 5   volt_sell         int64  
 6   pea_meter_number  object 
 7   kwh_pk            float64
 8   kwh_pp_op         float64
 9   kwh_op_h          float64
 10  kwh_total         float64
 11  amt_kwh_pk        float64
 12  amt_kwh_op        float64
 13  amt_kwh_tot       float64
dtypes: float64(7), int64(4), object(3)
memory usage: 230.4+ MB


### Data Cleansing

In [35]:
ca_dup = profile.ca.value_counts().reset_index()
ca_dup

,ca,count
0,20007378851,4745
1,20003661342,1357
2,20003655106,1226
3,20015866910,1104
4,20003655159,1099
...,...,...
182620,20027194946,1
182621,20027250447,1
182622,20027171619,1
182623,20027271087,1


In [36]:
profile.sort_values(by=['ca','period'],inplace=True)
profile.reset_index(drop=True,inplace=True)
profile

,trsg,period,ca,installation,ratecat,volt_sell,pea_meter_number,kwh_pk,kwh_pp_op,kwh_op_h,kwh_total,amt_kwh_pk,amt_kwh_op,amt_kwh_tot
0,G18101,202301,20000020082,6001131344,2114,4,6000549500,1890.5,773.0,1085.5,3749.0,0.0,0.0,14653.33
1,G18101,202302,20000020082,6001131344,2114,4,6000549500,2207.0,899.0,1247.0,4353.0,0.0,0.0,17014.14
2,G18101,202303,20000020082,6001131344,2114,4,6000549500,2525.5,1015.5,1669.0,5210.0,0.0,0.0,20363.81
3,G18101,202304,20000020082,6001131344,2114,4,6000549500,2555.0,1074.5,1735.5,5365.0,0.0,0.0,20969.63
4,G18101,202305,20000020082,6001131344,2114,4,6000549500,2604.0,1147.5,2328.5,6080.0,0.0,0.0,23764.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2156775,B02301,202312,20027333054,6018471478,2114,4,5701548121,256.0,176.8,0.0,432.8,0.0,0.0,1691.64
2156776,C11102,202312,20027333058,6025634426,2114,4,6001700655,6.0,6.0,0.0,12.0,0.0,0.0,46.90
2156777,A05201,202312,20027333389,6025634624,2114,4,6400037102,8.8,3.2,16.8,28.8,0.0,0.0,112.57
2156778,G02302,202312,20027333411,6025634508,2114,4,6400046813,436.8,235.2,292.0,964.0,0.0,0.0,3767.89


In [37]:
profile[profile.ca==20007378851]

,trsg,period,ca,installation,ratecat,volt_sell,pea_meter_number,kwh_pk,kwh_pp_op,kwh_op_h,kwh_total,amt_kwh_pk,amt_kwh_op,amt_kwh_tot
606967,L01101,202301,20007378851,6005666391,6124S,4,27731214,3360.09,1301.52,872.39,5534.00,0.0,0.0,0.0
606968,L01101,202301,20007378851,6005670379,6124S,4,6400256114,3257.00,0.00,0.00,3257.00,0.0,0.0,0.0
606969,L01101,202301,20007378851,6005670412,6124S,4,5800312534,1295.00,0.00,0.00,1295.00,0.0,0.0,0.0
606970,L01101,202301,20007378851,6005670432,6124S,4,30067609,1958.00,0.00,0.00,1958.00,0.0,0.0,0.0
606971,L01101,202301,20007378851,6005670504,6124S,4,NaN,0.00,0.00,0.00,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611707,L01114,202312,20007378851,6005969169,6124S,4,6600129875,1798.26,0.00,0.00,1798.26,0.0,0.0,0.0
611708,L01114,202312,20007378851,6006085728,6124S,4,6300752233,448.80,0.00,0.00,448.80,0.0,0.0,0.0
611709,L01114,202312,20007378851,6006085729,6124S,4,27875959,2795.82,0.00,0.00,2795.82,0.0,0.0,0.0
611710,L01114,202312,20007378851,6006098810,6124S,4,30072621,1112.82,0.00,0.00,1112.82,0.0,0.0,0.0


In [38]:
profile[profile.ca==20003661342]

,trsg,period,ca,installation,ratecat,volt_sell,pea_meter_number,kwh_pk,kwh_pp_op,kwh_op_h,kwh_total,amt_kwh_pk,amt_kwh_op,amt_kwh_tot
414818,L03103,202301,20003661342,6005763976,6124S,4,21112056,508.00,0.0,0.0,508.00,0.0,0.0,0.0
414819,L03103,202301,20003661342,6005763991,6124S,4,26946768,552.00,0.0,0.0,552.00,0.0,0.0,0.0
414820,L03103,202301,20003661342,6005764046,6124S,4,26361800,412.00,0.0,0.0,412.00,0.0,0.0,0.0
414821,L03103,202301,20003661342,6005764571,6124S,4,26946770,605.00,0.0,0.0,605.00,0.0,0.0,0.0
414822,L03103,202301,20003661342,6005802185,6124S,4,23360322,609.00,0.0,0.0,609.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416170,L03103,202312,20003661342,6023316330,6124S,4,6300783359,529.38,0.0,0.0,529.38,0.0,0.0,0.0
416171,L03103,202312,20003661342,6023316336,6124S,4,6300783360,0.00,0.0,0.0,0.00,0.0,0.0,0.0
416172,L03103,202312,20003661342,6023316338,6124S,4,6300783361,763.98,0.0,0.0,763.98,0.0,0.0,0.0
416173,L03103,202312,20003661342,6023549469,6124S,4,6300875525,518.16,0.0,0.0,518.16,0.0,0.0,0.0


In [39]:
profile[profile.ca==20027194946]

,trsg,period,ca,installation,ratecat,volt_sell,pea_meter_number,kwh_pk,kwh_pp_op,kwh_op_h,kwh_total,amt_kwh_pk,amt_kwh_op,amt_kwh_tot
2155919,A06101,202312,20027194946,6025519328,2114,4,6600077986,377.0,0.0,0.0,377.0,0.0,0.0,1473.55


### merge ca

In [40]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2156780 entries, 0 to 2156779
Data columns (total 14 columns):
 #   Column            Dtype  
---  ------            -----  
 0   trsg              object 
 1   period            int64  
 2   ca                int64  
 3   installation      int64  
 4   ratecat           object 
 5   volt_sell         int64  
 6   pea_meter_number  object 
 7   kwh_pk            float64
 8   kwh_pp_op         float64
 9   kwh_op_h          float64
 10  kwh_total         float64
 11  amt_kwh_pk        float64
 12  amt_kwh_op        float64
 13  amt_kwh_tot       float64
dtypes: float64(7), int64(4), object(3)
memory usage: 230.4+ MB


In [41]:
col = ['kwh_pk','kwh_pp_op','kwh_op_h','kwh_total','amt_kwh_pk','amt_kwh_op','amt_kwh_tot']

In [42]:
mean_usage = profile.groupby(['ca','period'])[col].agg('mean').reset_index()
mean_usage

,ca,period,kwh_pk,kwh_pp_op,kwh_op_h,kwh_total,amt_kwh_pk,amt_kwh_op,amt_kwh_tot
0,20000020082,202301,1890.5,773.0,1085.5,3749.0,0.0,0.0,14653.33
1,20000020082,202302,2207.0,899.0,1247.0,4353.0,0.0,0.0,17014.14
2,20000020082,202303,2525.5,1015.5,1669.0,5210.0,0.0,0.0,20363.81
3,20000020082,202304,2555.0,1074.5,1735.5,5365.0,0.0,0.0,20969.63
4,20000020082,202305,2604.0,1147.5,2328.5,6080.0,0.0,0.0,23764.29
...,...,...,...,...,...,...,...,...,...
2105460,20027333054,202312,256.0,176.8,0.0,432.8,0.0,0.0,1691.64
2105461,20027333058,202312,6.0,6.0,0.0,12.0,0.0,0.0,46.90
2105462,20027333389,202312,8.8,3.2,16.8,28.8,0.0,0.0,112.57
2105463,20027333411,202312,436.8,235.2,292.0,964.0,0.0,0.0,3767.89


In [43]:
mean_usage[mean_usage.ca==20003661342]

,ca,period,kwh_pk,kwh_pp_op,kwh_op_h,kwh_total,amt_kwh_pk,amt_kwh_op,amt_kwh_tot
384159,20003661342,202301,391.663717,0.0,0.0,396.778761,0.0,0.0,0.000000
384160,20003661342,202302,354.327434,0.0,0.0,359.442478,0.0,0.0,0.000000
384161,20003661342,202303,377.472212,0.0,0.0,382.587257,0.0,0.0,0.000000
384162,20003661342,202304,363.923363,0.0,0.0,369.038407,0.0,0.0,0.000000
384163,20003661342,202305,359.816283,0.0,0.0,364.931327,0.0,0.0,0.000000
384164,20003661342,202306,346.484071,0.0,0.0,351.599115,0.0,0.0,0.000000
384165,20003661342,202307,341.420177,0.0,0.0,346.535221,0.0,0.0,0.000000
384166,20003661342,202308,351.313274,0.0,0.0,356.428319,0.0,0.0,0.000000
384167,20003661342,202309,351.130526,0.0,0.0,375.100877,0.0,0.0,73.873246
384168,20003661342,202310,359.978761,0.0,0.0,365.093805,0.0,0.0,0.000000


In [44]:
mean_usage.ca.value_counts()

,count
ca,
20000020082,12
20020384957,12
20020385125,12
20020385131,12
20020385134,12
...,...
20027052679,1
20027049839,1
20027049838,1


In [45]:
test = profile.copy()

In [46]:
test.drop_duplicates(subset='ca',inplace=True)
test.reset_index(drop=True,inplace=True)

# col = ['kwh_pk','kwh_pp_op','kwh_op_h','kwh_total','amt_kwh_pk','amt_kwh_op','amt_kwh_tot']
test.drop(columns=['kwh_pk','kwh_pp_op','kwh_op_h','kwh_total','amt_kwh_pk','amt_kwh_op','amt_kwh_tot','period'],inplace=True)

test

,trsg,ca,installation,ratecat,volt_sell,pea_meter_number
0,G18101,20000020082,6001131344,2114,4,6000549500
1,G21101,20000021030,6000489476,2114,4,23434966
2,G21101,20000021041,6000489484,2114,4,6300576185
3,G21101,20000021063,6000489493,2114,4,6300576175
4,G21101,20000021083,6000489501,2114,4,6300576176
...,...,...,...,...,...,...
182620,B02301,20027333054,6018471478,2114,4,5701548121
182621,C11102,20027333058,6025634426,2114,4,6001700655
182622,A05201,20027333389,6025634624,2114,4,6400037102
182623,G02302,20027333411,6025634508,2114,4,6400046813


In [47]:
pf = pd.merge(test,mean_usage,on=['ca'])
pf

,trsg,ca,installation,ratecat,volt_sell,pea_meter_number,period,kwh_pk,kwh_pp_op,kwh_op_h,kwh_total,amt_kwh_pk,amt_kwh_op,amt_kwh_tot
0,G18101,20000020082,6001131344,2114,4,6000549500,202301,1890.5,773.0,1085.5,3749.0,0.0,0.0,14653.33
1,G18101,20000020082,6001131344,2114,4,6000549500,202302,2207.0,899.0,1247.0,4353.0,0.0,0.0,17014.14
2,G18101,20000020082,6001131344,2114,4,6000549500,202303,2525.5,1015.5,1669.0,5210.0,0.0,0.0,20363.81
3,G18101,20000020082,6001131344,2114,4,6000549500,202304,2555.0,1074.5,1735.5,5365.0,0.0,0.0,20969.63
4,G18101,20000020082,6001131344,2114,4,6000549500,202305,2604.0,1147.5,2328.5,6080.0,0.0,0.0,23764.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2105460,B02301,20027333054,6018471478,2114,4,5701548121,202312,256.0,176.8,0.0,432.8,0.0,0.0,1691.64
2105461,C11102,20027333058,6025634426,2114,4,6001700655,202312,6.0,6.0,0.0,12.0,0.0,0.0,46.90
2105462,A05201,20027333389,6025634624,2114,4,6400037102,202312,8.8,3.2,16.8,28.8,0.0,0.0,112.57
2105463,G02302,20027333411,6025634508,2114,4,6400046813,202312,436.8,235.2,292.0,964.0,0.0,0.0,3767.89


In [48]:
pf.ca.value_counts()

,count
ca,
20000020082,12
20020384957,12
20020385125,12
20020385131,12
20020385134,12
...,...
20027052679,1
20027049839,1
20027049838,1


## 4. merge all

In [49]:
merge

,ca,first_mi_1,last_mo_1,last_sec_dep_cash_sum,last_sec_dep_noncash_sum,sum_sec_dep,cus_age,c_status,partition_index_x,period,...,TAX_ID,BRANCH,DEPT_DOC_TYPE,PAY_DOC_TYPE,PAYMENT_CH,BILL_PERIOD_F,BILL_PERIOD_D,BILL_DUEDATE_F,BILL_PAYDATE_F,partition_index_y
0,20000020082,2008-10-02,9999-12-31,64000.0,0.0,64000.0,15.0,active,0,202301,...,3140600319095,NaN,IN,PG,รับชำระเงิน กฟข ก1,2023-01-01,202301.0,2023-02-17,2023-02-14,0
1,20000020082,2008-10-02,9999-12-31,64000.0,0.0,64000.0,15.0,active,0,202302,...,3140600319095,NaN,IN,PG,รับชำระเงิน กฟข ก1,2023-02-01,202302.0,2023-03-17,2023-03-14,3
2,20000020082,2008-10-02,9999-12-31,64000.0,0.0,64000.0,15.0,active,0,202303,...,3140600319095,NaN,IN,PG,รับชำระเงิน กฟข ก1,2023-03-01,202303.0,2023-04-18,2023-04-10,3
3,20000020082,2008-10-02,9999-12-31,64000.0,0.0,64000.0,15.0,active,0,202304,...,3140600319095,NaN,IN,PG,รับชำระเงิน กฟข ก1,2023-04-01,202304.0,2023-05-18,2023-05-18,7
4,20000020082,2008-10-02,9999-12-31,64000.0,0.0,64000.0,15.0,active,0,202305,...,3140600319095,NaN,IN,PG,รับชำระเงิน กฟข ก1,2023-05-01,202305.0,2023-06-19,2023-06-16,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002111,20026328304,2023-01-22,9999-12-31,0.0,0.0,0.0,1.0,active,0,202308,...,0206101818752,00001,IN,MG,GFMIS - ใบกำกับฯ,2023-08-01,202308.0,2023-09-21,2023-11-03,6
2002112,20026328304,2023-01-22,9999-12-31,0.0,0.0,0.0,1.0,active,0,202309,...,0206101818752,00001,IN,MG,GFMIS - ใบกำกับฯ,2023-09-01,202309.0,2023-10-24,2023-12-04,5
2002113,20026328304,2023-01-22,9999-12-31,0.0,0.0,0.0,1.0,active,0,202310,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
2002114,20026328304,2023-01-22,9999-12-31,0.0,0.0,0.0,1.0,active,0,202311,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [50]:
pf

,trsg,ca,installation,ratecat,volt_sell,pea_meter_number,period,kwh_pk,kwh_pp_op,kwh_op_h,kwh_total,amt_kwh_pk,amt_kwh_op,amt_kwh_tot
0,G18101,20000020082,6001131344,2114,4,6000549500,202301,1890.5,773.0,1085.5,3749.0,0.0,0.0,14653.33
1,G18101,20000020082,6001131344,2114,4,6000549500,202302,2207.0,899.0,1247.0,4353.0,0.0,0.0,17014.14
2,G18101,20000020082,6001131344,2114,4,6000549500,202303,2525.5,1015.5,1669.0,5210.0,0.0,0.0,20363.81
3,G18101,20000020082,6001131344,2114,4,6000549500,202304,2555.0,1074.5,1735.5,5365.0,0.0,0.0,20969.63
4,G18101,20000020082,6001131344,2114,4,6000549500,202305,2604.0,1147.5,2328.5,6080.0,0.0,0.0,23764.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2105460,B02301,20027333054,6018471478,2114,4,5701548121,202312,256.0,176.8,0.0,432.8,0.0,0.0,1691.64
2105461,C11102,20027333058,6025634426,2114,4,6001700655,202312,6.0,6.0,0.0,12.0,0.0,0.0,46.90
2105462,A05201,20027333389,6025634624,2114,4,6400037102,202312,8.8,3.2,16.8,28.8,0.0,0.0,112.57
2105463,G02302,20027333411,6025634508,2114,4,6400046813,202312,436.8,235.2,292.0,964.0,0.0,0.0,3767.89


In [51]:
merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2002116 entries, 0 to 2002115
Data columns (total 37 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   ca                        int64  
 1   first_mi_1                object 
 2   last_mo_1                 object 
 3   last_sec_dep_cash_sum     float64
 4   last_sec_dep_noncash_sum  float64
 5   sum_sec_dep               float64
 6   cus_age                   float64
 7   c_status                  object 
 8   partition_index_x         int64  
 9   period                    int64  
 10  trsg                      object 
 11  installation              int64  
 12  amt_invoice               float64
 13  free                      object 
 14  discount_operand          object 
 15  min_waived                object 
 16  tmp_dsc                   object 
 17  OPBEL                     float64
 18  XBLNR                     float64
 19  CA_NAME                   object 
 20  KTOKL                   

In [52]:
pf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2105465 entries, 0 to 2105464
Data columns (total 14 columns):
 #   Column            Dtype  
---  ------            -----  
 0   trsg              object 
 1   ca                int64  
 2   installation      int64  
 3   ratecat           object 
 4   volt_sell         int64  
 5   pea_meter_number  object 
 6   period            int64  
 7   kwh_pk            float64
 8   kwh_pp_op         float64
 9   kwh_op_h          float64
 10  kwh_total         float64
 11  amt_kwh_pk        float64
 12  amt_kwh_op        float64
 13  amt_kwh_tot       float64
dtypes: float64(7), int64(4), object(3)
memory usage: 224.9+ MB


In [96]:
col = [
    'ca',
    'period',
    # 'trsg',
    # 'installation'
    ]

In [97]:
credit = pd.merge(merge,pf,on=col)

credit

,ca,first_mi_1,last_mo_1,last_sec_dep_cash_sum,last_sec_dep_noncash_sum,sum_sec_dep,cus_age,c_status,partition_index_x,period,...,ratecat,volt_sell,pea_meter_number,kwh_pk,kwh_pp_op,kwh_op_h,kwh_total,amt_kwh_pk,amt_kwh_op,amt_kwh_tot
0,20000020082,2008-10-02,9999-12-31,64000.0,0.0,64000.0,15.0,active,0,202301,...,2114,4,6000549500,1890.5,773.0,1085.5,3749.0,0.0,0.0,14653.33
1,20000020082,2008-10-02,9999-12-31,64000.0,0.0,64000.0,15.0,active,0,202302,...,2114,4,6000549500,2207.0,899.0,1247.0,4353.0,0.0,0.0,17014.14
2,20000020082,2008-10-02,9999-12-31,64000.0,0.0,64000.0,15.0,active,0,202303,...,2114,4,6000549500,2525.5,1015.5,1669.0,5210.0,0.0,0.0,20363.81
3,20000020082,2008-10-02,9999-12-31,64000.0,0.0,64000.0,15.0,active,0,202304,...,2114,4,6000549500,2555.0,1074.5,1735.5,5365.0,0.0,0.0,20969.63
4,20000020082,2008-10-02,9999-12-31,64000.0,0.0,64000.0,15.0,active,0,202305,...,2114,4,6000549500,2604.0,1147.5,2328.5,6080.0,0.0,0.0,23764.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002111,20026328304,2023-01-22,9999-12-31,0.0,0.0,0.0,1.0,active,0,202308,...,2114,4,6223236,0.0,0.0,0.0,0.0,0.0,0.0,0.00
2002112,20026328304,2023-01-22,9999-12-31,0.0,0.0,0.0,1.0,active,0,202309,...,2114,4,6223236,0.0,0.0,0.0,0.0,0.0,0.0,0.00
2002113,20026328304,2023-01-22,9999-12-31,0.0,0.0,0.0,1.0,active,0,202310,...,2114,4,6223236,0.0,0.0,0.0,0.0,0.0,0.0,0.00
2002114,20026328304,2023-01-22,9999-12-31,0.0,0.0,0.0,1.0,active,0,202311,...,2114,4,6223236,0.0,0.0,0.0,0.0,0.0,0.0,0.00


In [98]:
credit.ca.value_counts()

,count
ca,
20000020082,12
20020435479,12
20020433513,12
20020433533,12
20020433535,12
...,...
20009581014,12
20009581041,12
20009581047,12


In [99]:
credit[credit.ca==20027333054]

,ca,first_mi_1,last_mo_1,last_sec_dep_cash_sum,last_sec_dep_noncash_sum,sum_sec_dep,cus_age,c_status,partition_index_x,period,...,ratecat,volt_sell,pea_meter_number,kwh_pk,kwh_pp_op,kwh_op_h,kwh_total,amt_kwh_pk,amt_kwh_op,amt_kwh_tot


In [100]:
credit[credit.ca==20014289012]

,ca,first_mi_1,last_mo_1,last_sec_dep_cash_sum,last_sec_dep_noncash_sum,sum_sec_dep,cus_age,c_status,partition_index_x,period,...,ratecat,volt_sell,pea_meter_number,kwh_pk,kwh_pp_op,kwh_op_h,kwh_total,amt_kwh_pk,amt_kwh_op,amt_kwh_tot
810216,20014289012,2010-10-30,9999-12-31,40000.0,0.0,40000.0,13.0,active,0,202301,...,2114,4,6101486180,699.0,448.5,460.5,1608.0,0.0,0.0,6285.03
810217,20014289012,2010-10-30,9999-12-31,40000.0,0.0,40000.0,13.0,active,0,202302,...,2114,4,6101486180,939.9,580.2,500.7,2020.8,0.0,0.0,7898.50
810218,20014289012,2010-10-30,9999-12-31,40000.0,0.0,40000.0,13.0,active,0,202303,...,2114,4,6101486180,1214.4,636.3,743.7,2594.4,0.0,0.0,10140.47
810219,20014289012,2010-10-30,9999-12-31,40000.0,0.0,40000.0,13.0,active,0,202304,...,2114,4,6101486180,1658.1,666.6,981.6,3306.3,0.0,0.0,12923.01
810220,20014289012,2010-10-30,9999-12-31,40000.0,0.0,40000.0,13.0,active,0,202305,...,2114,4,6101486180,1569.6,708.3,1166.1,3444.0,0.0,0.0,13461.22
810221,20014289012,2010-10-30,9999-12-31,40000.0,0.0,40000.0,13.0,active,0,202306,...,2114,4,6101486180,1483.8,690.0,727.2,2901.0,0.0,0.0,11338.85
810222,20014289012,2010-10-30,9999-12-31,40000.0,0.0,40000.0,13.0,active,0,202307,...,2114,4,6101486180,1377.3,604.5,660.9,2642.7,0.0,0.0,10329.26
810223,20014289012,2010-10-30,9999-12-31,40000.0,0.0,40000.0,13.0,active,0,202308,...,2114,4,6101486180,1378.2,535.2,944.4,2857.8,0.0,0.0,11170.00
810224,20014289012,2010-10-30,9999-12-31,40000.0,0.0,40000.0,13.0,active,0,202309,...,2114,4,6101486180,1104.9,575.1,526.8,2206.8,0.0,0.0,8625.50
810225,20014289012,2010-10-30,9999-12-31,40000.0,0.0,40000.0,13.0,active,0,202310,...,2114,4,6101486180,990.3,501.0,677.7,2169.0,0.0,0.0,8477.75


In [101]:
outer = pd.merge(merge,pf,on=col,how='outer')
outer

,ca,first_mi_1,last_mo_1,last_sec_dep_cash_sum,last_sec_dep_noncash_sum,sum_sec_dep,cus_age,c_status,partition_index_x,period,...,ratecat,volt_sell,pea_meter_number,kwh_pk,kwh_pp_op,kwh_op_h,kwh_total,amt_kwh_pk,amt_kwh_op,amt_kwh_tot
0,20000020082,2008-10-02,9999-12-31,64000.0,0.0,64000.0,15.0,active,0.0,202301,...,2114,4,6000549500,1890.5,773.0,1085.5,3749.0,0.0,0.0,14653.33
1,20000020082,2008-10-02,9999-12-31,64000.0,0.0,64000.0,15.0,active,0.0,202302,...,2114,4,6000549500,2207.0,899.0,1247.0,4353.0,0.0,0.0,17014.14
2,20000020082,2008-10-02,9999-12-31,64000.0,0.0,64000.0,15.0,active,0.0,202303,...,2114,4,6000549500,2525.5,1015.5,1669.0,5210.0,0.0,0.0,20363.81
3,20000020082,2008-10-02,9999-12-31,64000.0,0.0,64000.0,15.0,active,0.0,202304,...,2114,4,6000549500,2555.0,1074.5,1735.5,5365.0,0.0,0.0,20969.63
4,20000020082,2008-10-02,9999-12-31,64000.0,0.0,64000.0,15.0,active,0.0,202305,...,2114,4,6000549500,2604.0,1147.5,2328.5,6080.0,0.0,0.0,23764.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2105460,20027333054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202312,...,2114,4,5701548121,256.0,176.8,0.0,432.8,0.0,0.0,1691.64
2105461,20027333058,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202312,...,2114,4,6001700655,6.0,6.0,0.0,12.0,0.0,0.0,46.90
2105462,20027333389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202312,...,2114,4,6400037102,8.8,3.2,16.8,28.8,0.0,0.0,112.57
2105463,20027333411,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202312,...,2114,4,6400046813,436.8,235.2,292.0,964.0,0.0,0.0,3767.89


In [102]:
credit.to_csv(
    '/content/drive/MyDrive/PEA-intern/CreditScore/credit_customers02.csv',
    index=False
)